In [ ]:
# --- Базовые библиотеки ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import collections # Для dm_test
from itertools import combinations # Для dm_test

# --- Метрики и Утилиты ---
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler # Оставим оба на всякий случай
from scipy.stats import t # Для dm_test
from statsmodels.sandbox.stats.multicomp import multipletests # Для DM-теста

# --- Модели Временных Рядов ---
from pmdarima.arima import auto_arima
from statsmodels.tsa.arima.model import ARIMA as StatsmodelsARIMA

# --- Модели Машинного Обучения ---
from sklearn.linear_model import Ridge, Lasso, RidgeCV, LassoCV
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

# --- (Опционально) Нейронные Сети ---
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers

# --- Настройки Визуализации и Предупреждений ---
plt.style.use('seaborn-v0_8-whitegrid') # Стиль графиков
warnings.filterwarnings("ignore") # Игнорировать предупреждения (можно закомментировать для отладки)

print("Библиотеки импортированы.")

In [ ]:
# --- Глобальные Настройки ---
RANDOM_SEED = 42 # Для воспроизводимости
np.random.seed(RANDOM_SEED)
# tf.random.set_seed(RANDOM_SEED) # Если используем TensorFlow

# --- Параметры Данных ---
FILE_PATH = 'CPI.xlsx' 
WEIGHTS_FILE_PATH = 'Weights.csv'
CPI_MONTHLY_SHEET = '44 компоненты' 
WEIGHTS_SHEET = 'Веса'
DATE_COLUMN = 'Дата'
CPI_TOTAL_COLUMN = 'ИПЦ'
COMPONENTS_TO_MODEL = ['Мясопродукты', 'Рыбопродукты', 'Масло и жиры',
       'Молоко и молочная продукция', 'Сыр', 'Яйца', 'Сахар',
       'Кондитерские изделия', 'Чай, кофе',
       'Хлеб и хлебобулочные изделия', 'Макаронные и крупяные изделия',
       'Плодоовощная продукция, включая картофель', 'Алкогольные напитки',
       'Общественное питание', 'Одежда и белье', 'Меха и меховые изделия',
       'Трикотажные изделия',
       'Обувь кожаная, текстильная и комбинированная',
       'Моющие и чистящие средства', 'Парфюмерно-косметические товары',
       'Галантерея', 'Табачные изделия', 'Мебель',
       'Электротовары и другие бытовые приборы', 'Печатные издания',
       'Телерадиотовары', 'Персональные компьютеры', 'Средства связи',
       'Строительные материалы', 'Легковые автомобили',
       'Инструменты и оборудование', 'Нефтепродукты',
       'Медицинские товары', 'Бытовые услуги',
       'Услуги пассажирского транспорта', 'Услуги связи',
       'Жилищно-коммунальные услуги',
       'Услуги гостиниц и прочих мест проживания',
       'Услуги в системе образования', 'Услуги организаций культуры',
       'Услуги в сфере зарубежного туризма', 'Экскурсионные услуги',
       'Санаторно-оздоровительные услуги', 'Медицинские услуги']

# --- Параметры Времени и Оценки ---
TEST_MONTHS = 24
TRAIN_DO = '2023-02-01' # Дата начала тестового периода
HORIZONS_TO_EVALUATE = [1, 2, 3, 6, 12, 18, 24] # Горизонты для расчета RMSE
PLOT_START_DATE = '2016-01-01' # Начальная дата для графиков прогнозов

# --- Параметры Кросс-Валидации и Feature Engineering ---
N_CV_SPLITS = 3 # Количество сплитов для GridSearchCV при обучении моделей компонент
N_OOF_SPLITS = 5 # Количество сплитов для генерации OOF прогнозов
LAGS_TO_CREATE = [1, 2, 3, 6, 12] # Лаги для признаков ML
ROLLING_WINDOWS = [3, 6, 12] # Окна для скользящих статистик ML
# Порядок признаков (важно для функций)
FEATURE_ORDER = [
    't-1', 't-2', 't-3', 't-6', 't-12',
    't_mean_lag3', 't_std_lag3',
    't_mean_lag6', 't_std_lag6',
    't_mean_lag12', 't_std_lag12',
    'month_sin', 'month_cos'
]

# --- Параметры Критерия Выбора Лучшей Модели Компоненты ---
HORIZONS_FOR_WEIGHTING = [1, 2, 3, 6, 12, 18, 24] # Горизонты для взвешенного RMSE
weights_raw = pd.Series({h: 1/np.sqrt(h) for h in HORIZONS_FOR_WEIGHTING})
WEIGHTS_NORMALIZED_RMSE = weights_raw / weights_raw.sum()
print("Веса для выбора лучшей модели компоненты:")
print(WEIGHTS_NORMALIZED_RMSE.round(4))

# --- Инициализация Глобальных Структур для Результатов ---
# Общие результаты агрегации
aggregation_rmse = {} # Словарь: {имя_агрег_модели: общий_rmse}
all_horizon_rmse = pd.DataFrame(columns=['model', 'horizon', 'rmse']) # RMSE по горизонтам для агрег. моделей

# Результаты по компонентам
component_best_models = {} # Словарь: {компонента: имя_лучшей_модели}
df_best_component_forecasts = None # DataFrame: [ИндексТеста x Компоненты] - прогнозы ЛУЧШИХ моделей на тесте
df_all_component_rmses = None # DataFrame: [Компонента, Горизонт, Модель] - RMSE всех моделей для всех компонент

# Промежуточные прогнозы компонент
df_forecasts_arima_comp = None # DataFrame: [ИндексТеста x Компоненты] - прогнозы ARIMA на тесте
df_oof_comp_arima_clean = None # DataFrame: [ИндексТрейна x Компоненты] - OOF прогнозы ARIMA (чистые)
df_oof_best_comp_clean = None # DataFrame: [ИндексТрейна x Компоненты] - OOF прогнозы ЛУЧШИХ моделей (чистые)

# Финальные прогнозы для графиков и DM-тестов
final_forecasts = {} # Словарь: {имя_финальной_модели: np.array прогноза на тест}

print("\nНастройки и структуры результатов инициализированы (с CV для компонент).")

In [ ]:
# df_monthly = pd.read_excel(FILE_PATH,
#                                engine='openpyxl',
#                                sheet_name=CPI_MONTHLY_SHEET)
# df_monthly

In [ ]:
# df_monthly.info()

In [ ]:
# df_weights_raw = pd.read_csv(WEIGHTS_FILE_PATH)
# df_weights_raw[df_weights_raw['Уровень'] == '44 компоненты']['Компонента'].unique()

In [ ]:
# df_weights_raw

In [ ]:
print("--- Раздел 1: Загрузка и Подготовка Данных ---")
print("\nЯчейка 3: Загрузка ежемесячных данных (ИПЦ + Компоненты)")

try:
    # Загружаем лист с ежемесячными данными
    df_monthly = pd.read_excel(FILE_PATH,
                               engine='openpyxl',
                               sheet_name=CPI_MONTHLY_SHEET) # Используем имя листа из настроек

    # Преобразование колонки с датой
    df_monthly[DATE_COLUMN] = pd.to_datetime(df_monthly[DATE_COLUMN])
    df_monthly = df_monthly.set_index(DATE_COLUMN)

    # Выбираем только нужные колонки: общий ИПЦ и заданные компоненты
    columns_to_keep = [CPI_TOTAL_COLUMN] + COMPONENTS_TO_MODEL
    df_monthly = df_monthly[columns_to_keep]

    # # Удаление строк с пропусками, если есть
    # initial_rows = len(df_monthly)
    # df_monthly = df_monthly.dropna()
    # final_rows = len(df_monthly)
    # if initial_rows > final_rows:
    #     print(f"Удалено {initial_rows - final_rows} строк с NaN.")

    print(f"\nЕжемесячные данные загружены и обработаны.")
    print(f"Период данных: {df_monthly.index.min().strftime('%Y-%m-%d')} - {df_monthly.index.max().strftime('%Y-%m-%d')}")
    print(f"Размер DataFrame: {df_monthly.shape}")
    print("\nПервые 5 строк:")
    print(df_monthly.head())
    print("\nПоследние 5 строк:")
    print(df_monthly.tail())
    print("\nИнформация о данных:")
    df_monthly.info()

except FileNotFoundError:
    print(f"ОШИБКА: Файл не найден по пути: {FILE_PATH}")
    # Выход или дальнейшие действия в зависимости от логики ноутбука
    raise # Поднимаем ошибку, чтобы остановить выполнение, если файл критичен
except KeyError as e:
    print(f"ОШИБКА: Не найдена колонка {e} на листе '{CPI_MONTHLY_SHEET}'. Проверьте настройки CPI_TOTAL_COLUMN и COMPONENTS_TO_MODEL.")
    raise
except Exception as e:
    print(f"ОШИБКА при загрузке или обработке листа '{CPI_MONTHLY_SHEET}': {e}")
    raise

In [ ]:
print("\nЯчейка 3а: Анализ и Заполнение Пропусков в Компонентах")

# --- ПРЕДПОЛАГАЕМ НАЛИЧИЕ ПЕРЕМЕННЫХ ---
# df_monthly: DataFrame с ИПЦ и всеми компонентами (после Ячейки 3)
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

# Компоненты с пропусками для анализа
cols_to_impute = ['Персональные компьютеры', 'Средства связи']
col_to_exclude = 'Инструменты и оборудование' # Компонента для исключения

# --- 1. Исключение компоненты ---
if col_to_exclude in df_monthly.columns:
    df_monthly = df_monthly.drop(columns=[col_to_exclude])
    print(f"Компонента '{col_to_exclude}' исключена из DataFrame.")
    # Удаляем ее из глобального списка компонент, если он уже определен
    if 'COMPONENTS_TO_MODEL' in globals() and isinstance(COMPONENTS_TO_MODEL, list):
         if col_to_exclude in COMPONENTS_TO_MODEL:
             COMPONENTS_TO_MODEL.remove(col_to_exclude)
             print(f"'{col_to_exclude}' удалена из списка COMPONENTS_TO_MODEL.")
else:
    print(f"Компонента '{col_to_exclude}' не найдена в df_monthly.")


# --- 2. Анализ и Визуализация рядов ДО заполнения ---
print("\nАнализ рядов с пропусками ДО заполнения:")
missing_data_info = df_monthly[cols_to_impute].isnull().sum()
print("Количество пропусков:")
print(missing_data_info)

# Строим графики
plt.figure(figsize=(14, 5 * len(cols_to_impute)))
for i, col in enumerate(cols_to_impute):
    if col in df_monthly.columns:
        plt.subplot(len(cols_to_impute), 1, i + 1)
        plt.plot(df_monthly.index, df_monthly[col], label=f'{col} (Original)', marker='.', linestyle='-')
        plt.title(f'Компонента: {col} (До заполнения)')
        plt.ylabel('Значение')
        plt.grid(True)
        plt.legend()
    else:
         print(f"Колонка '{col}' не найдена для визуализации.")
plt.xlabel('Дата')
plt.tight_layout()
plt.show()

# --- 3. Заполнение пропусков линейной интерполяцией ---
print("\nЗаполнение пропусков линейной интерполяцией...")
df_monthly_imputed = df_monthly.copy() # Создаем копию для изменений

for col in cols_to_impute:
    if col in df_monthly_imputed.columns:
        initial_nan = df_monthly_imputed[col].isnull().sum()
        if initial_nan > 0:
            # limit_direction='both' заполняет NaN в начале и в конце
            df_monthly_imputed[col] = df_monthly_imputed[col].interpolate(method='linear', limit_direction='both')
            filled_nan = initial_nan - df_monthly_imputed[col].isnull().sum()
            print(f"  Для '{col}': Заполнено {filled_nan} из {initial_nan} пропусков.")
        else:
            print(f"  Для '{col}': Пропусков не найдено.")
    else:
        print(f"  Колонка '{col}' не найдена для заполнения.")

# Перезаписываем исходный DataFrame или используем новый
df_monthly = df_monthly_imputed # Перезаписываем df_monthly
print("\nПроверка NaN ПОСЛЕ интерполяции:")
print(df_monthly[cols_to_impute].isnull().sum())

# --- 4. Визуализация ПОСЛЕ интерполяции ---
print("\nВизуализация рядов ПОСЛЕ заполнения:")
plt.figure(figsize=(14, 5 * len(cols_to_impute)))
for i, col in enumerate(cols_to_impute):
     if col in df_monthly.columns:
        plt.subplot(len(cols_to_impute), 1, i + 1)
        # Отметим интерполированные значения другим цветом/маркером
        original_data_mask = df_monthly_imputed[col].notna() & df_monthly[col].isna() # Где были NaN
        interpolated_values = df_monthly[col].copy()
        interpolated_values[~original_data_mask] = np.nan # Оставляем только интерполированные

        plt.plot(df_monthly.index, df_monthly[col], label=f'{col} (Заполненный)', marker='.', linestyle='-', zorder=1)
        #plt.scatter(interpolated_values.index, interpolated_values, color='red', s=15, label='Интерполяция', zorder=2) # Отметим красным
        plt.title(f'Компонента: {col} (После заполнения)')
        plt.ylabel('Значение')
        plt.grid(True)
        plt.legend()
     else:
          print(f"Колонка '{col}' не найдена для визуализации.")

plt.xlabel('Дата')
plt.tight_layout()
plt.show()

print("\n--- Ячейка 3а завершена ---")

In [ ]:
df_monthly.info()

In [ ]:
print("\nЯчейка 4: Загрузка и Обработка Данных по Годовым Весам")

try:
    # Загружаем лист с весами
    df_weights_raw = pd.read_csv(WEIGHTS_FILE_PATH) # Используем имя листа из настроек

    print("Структура исходного файла весов (первые 5 строк):")
    print(df_weights_raw.head())

    # --- Адаптация под структуру файла весов ---
    # Предполагаем структуру: 'Уровень', 'Компонента', Год1, Год2, ...
    # Отбираем строки, относящиеся к 3 компонентам
    # !!! Убедись, что значение '3 компоненты' точное !!!
    df_weights_filtered = df_weights_raw[df_weights_raw['Уровень'] == '44 компоненты'].copy()

    if df_weights_filtered.empty:
        raise ValueError(f"Не найдены строки с 'Уровень' == '6 компонент' на листе '{WEIGHTS_SHEET}'.")

    # Убираем ненужную колонку "Уровень" и делаем компоненты индексом
    df_weights_processed = df_weights_filtered.drop(columns=['Уровень']).set_index('Компонента')

    # Транспонируем, чтобы годы стали индексом
    weights_df_transposed = df_weights_processed.T

    # Преобразуем индекс (годы) в числовой формат и проверяем
    weights_df_transposed.index = pd.to_numeric(weights_df_transposed.index, errors='coerce')
    weights_df_transposed = weights_df_transposed.dropna(axis=0) # Удаляем строки, если год не распознался
    weights_df_transposed.index = weights_df_transposed.index.astype(int)
    weights_df_transposed.index.name = 'Год'

    # Убираем лишние пробелы в названиях колонок (на всякий случай)
    weights_df_transposed.columns = weights_df_transposed.columns.str.strip()

    # Выбираем и УПОРЯДОЧИВАЕМ колонки согласно COMPONENTS_TO_MODEL
    weights_df_pivot = weights_df_transposed[COMPONENTS_TO_MODEL]

    print("\nДанные по весам успешно обработаны (Год x Компонента):")
    print(weights_df_pivot.head())
    print(weights_df_pivot.tail())

    # Нормализуем веса (делим на 100, чтобы сумма стала 1)
    weights_norm = weights_df_pivot.apply(lambda x: x / 100.0, axis=1)
    print("\nНормализованные веса (доли):")
    print(weights_norm.head())
    sum_weights_modelled = weights_norm.sum(axis=1)
    print("\nСуммарный вес моделируемых компонент по годам (первые 5 лет):")
    print(sum_weights_modelled.head())
    print(f"\nСредний суммарный вес за весь период: {sum_weights_modelled.mean():.4f}")

except FileNotFoundError:
    print(f"ОШИБКА: Файл не найден по пути: {FILE_PATH}")
    raise
except KeyError as e:
     print(f"ОШИБКА: Не найдена колонка {e} при обработке весов. Проверьте названия 'Уровень', 'Компонента' и годов на листе '{WEIGHTS_SHEET}'.")
     raise
except ValueError as e:
     print(f"ОШИБКА: {e}")
     raise
except Exception as e:
    print(f"ОШИБКА при загрузке или обработке листа весов '{WEIGHTS_SHEET}': {e}")
    raise

In [ ]:
print("\nЯчейка 5: Разделение Данных на Train/Test")

# --- Агрегированный ряд ---
try:
    cpi_agg_series = df_monthly[CPI_TOTAL_COLUMN]

    # Определяем дату среза из настроек
    train_do_dt = pd.to_datetime(TRAIN_DO)

    # Разделяем
    train_values = cpi_agg_series[cpi_agg_series.index < train_do_dt]
    test_values = cpi_agg_series[cpi_agg_series.index >= train_do_dt][:24]

    # Проверяем длину тестовой выборки
    if len(test_values) != TEST_MONTHS:
        print(f"ВНИМАНИЕ: Длина тестовой выборки агрегата ({len(test_values)}) не равна {TEST_MONTHS}.")
        # Можно добавить логику корректировки или остановки, если это критично
        # Например, взять последние TEST_MONTHS точек:
        # test_values = cpi_agg_series.iloc[-TEST_MONTHS:]
        # train_values = cpi_agg_series.iloc[:-TEST_MONTHS]
        # print(f"Скорректировано: Длина теста = {len(test_values)}, Трейна = {len(train_values)}")
        # Пока оставим как есть, но выведем предупреждение.

    print("\nАгрегированный ряд:")
    print(f"  Размер обучающей выборки (train_values): {len(train_values)} (до {train_values.index.max().strftime('%Y-%m-%d')})")
    print(f"  Размер тестовой выборки (test_values): {len(test_values)} (с {test_values.index.min().strftime('%Y-%m-%d')})")

except KeyError:
    print(f"ОШИБКА: Колонка общего ИПЦ '{CPI_TOTAL_COLUMN}' не найдена.")
    raise
except Exception as e:
     print(f"ОШИБКА при разделении агрегированного ряда: {e}")
     raise

# --- Компонентные ряды ---
try:
    cpi_components_data = df_monthly[COMPONENTS_TO_MODEL]

    # Разделяем по той же дате
    train_comp_data = cpi_components_data[cpi_components_data.index < train_do_dt]
    test_comp_data = cpi_components_data[cpi_components_data.index >= train_do_dt][:24]

    # Проверяем длину теста для компонент
    if len(test_comp_data) != TEST_MONTHS:
         print(f"ВНИМАНИЕ: Длина тестовой выборки компонент ({len(test_comp_data)}) не равна {TEST_MONTHS}.")
         # Аналогично, можно скорректировать или оставить с предупреждением.

    print("\nКомпонентные ряды:")
    print(f"  Размер обучающей выборки (train_comp_data): {train_comp_data.shape}")
    print(f"  Размер тестовой выборки (test_comp_data): {test_comp_data.shape}")

except KeyError:
    print(f"ОШИБКА: Не все колонки компонент {COMPONENTS_TO_MODEL} найдены.")
    raise
except Exception as e:
     print(f"ОШИБКА при разделении компонентных рядов: {e}")
     raise

print("\n--- Раздел 1 завершен ---")

In [ ]:
print("\n--- Раздел 2: Базовая Модель - ARIMA на Агрегате ---")
print("\nЯчейка 6: Обучение и Оценка Baseline ARIMA")

# --- ПРЕДПОЛАГАЕМ НАЛИЧИЕ ПЕРЕМЕННЫХ ---
# train_values: pd.Series - Обучающая выборка агрегированного ИПЦ
# test_values: pd.Series - Тестовая выборка агрегированного ИПЦ
# TEST_MONTHS: int - Количество месяцев в тесте
# HORIZONS_TO_EVALUATE: list - Горизонты для оценки RMSE
# aggregation_rmse: dict - Словарь для общих RMSE
# all_horizon_rmse: pd.DataFrame - DataFrame для RMSE по горизонтам
# final_forecasts: dict - Словарь для финальных прогнозов
# PLOT_START_DATE: str - Дата для начала графика
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

model_name_baseline = "Baseline ARIMA"

# 1. Подбор и обучение модели auto_arima
print(f"\nПодбор и обучение {model_name_baseline}...")
try:
    # Используем параметры, которые хорошо себя показали ранее
    baseline_arima_model = auto_arima(train_values,
                                      start_p=1, start_q=1, max_p=5, max_q=5, # Ограничим немного поиск
                                      m=1, seasonal=False, # Без сезонности для месячных приростов
                                      d=0,
                                      test='adf',       # Тест для d
                                      trace=False,      # Не выводить шаги подбора
                                      error_action='ignore',
                                      suppress_warnings=True,
                                      stepwise=True,    # Ускоренный подбор
                                      information_criterion='aic', # Критерий выбора
                                      n_jobs=-1) # Используем все ядра CPU

    print(f"Выбрана модель: {baseline_arima_model.order} {baseline_arima_model.seasonal_order}")

    # 2. Генерация рекурсивного прогноза
    print(f"Генерация рекурсивного прогноза на {TEST_MONTHS} шагов...")
    forecast_baseline_arima = baseline_arima_model.predict(n_periods=TEST_MONTHS)
    # Сохраняем прогноз в общий словарь
    final_forecasts[model_name_baseline] = forecast_baseline_arima[:TEST_MONTHS]
    print("Прогноз сохранен.")

    # 3. Оценка RMSE
    # Убедимся, что длина прогноза совпадает с тестом
    eval_len = min(len(forecast_baseline_arima), len(test_values))
    if eval_len < TEST_MONTHS:
        print(f"ВНИМАНИЕ: Длина прогноза ({eval_len}) меньше длины теста ({len(test_values)}). Оценка по {eval_len} точкам.")
    y_test_eval = test_values[:eval_len]
    forecast_eval = forecast_baseline_arima[:eval_len]

    overall_rmse_baseline = np.sqrt(mean_squared_error(y_test_eval, forecast_eval))
    print(f"\nОбщий RMSE на тесте для {model_name_baseline}: {overall_rmse_baseline:.4f}")

    # Добавляем/Обновляем общий RMSE
    aggregation_rmse[model_name_baseline] = overall_rmse_baseline

    # Расчет и добавление RMSE по горизонтам
    print("RMSE по горизонтам:")
    # Удаляем старые записи для этой модели, если есть
    all_horizon_rmse = all_horizon_rmse[all_horizon_rmse['model'] != model_name_baseline]
    horizon_results_list_baseline = []
    for h in HORIZONS_TO_EVALUATE:
        if h <= eval_len:
            rmse_h = np.sqrt(mean_squared_error(y_test_eval[:h], forecast_eval[:h]))
            print(f"  1-{h} мес.: {rmse_h:.4f}")
            horizon_results_list_baseline.append({'model': model_name_baseline, 'horizon': h, 'rmse': rmse_h})
        else:
             print(f"  1-{h} мес.: Недостаточно данных для расчета.")

    if horizon_results_list_baseline:
        df_horizon_rmse_baseline = pd.DataFrame(horizon_results_list_baseline)
        all_horizon_rmse = pd.concat([all_horizon_rmse, df_horizon_rmse_baseline], ignore_index=True)
    print("Результаты RMSE по горизонтам добавлены/обновлены.")

    # 4. Графики (опционально, но полезно)
    # График прогноза
    plt.figure(figsize=(12, 6))
    history_to_plot = train_values[train_values.index >= PLOT_START_DATE]
    plt.plot(history_to_plot.index, history_to_plot, label='Обучающая выборка (часть)', alpha=0.7)
    plt.plot(test_values.index, test_values, label='Тестовая выборка (реальные)', color='orange', linewidth=2)
    forecast_baseline_series = pd.Series(forecast_eval, index=test_values.index[:eval_len])
    plt.plot(forecast_baseline_series.index, forecast_baseline_series, label=f'Прогноз {model_name_baseline}', color='blue', linestyle='--')
    plt.title(f'Прогноз ИПЦ с помощью {model_name_baseline}')
    plt.xlabel('Дата'); plt.ylabel('ИПЦ (месячный рост)'); plt.legend(); plt.grid(True)
    plt.show()

    # График RMSE vs Горизонт
    if horizon_results_list_baseline:
        plt.figure(figsize=(10, 5))
        plt.plot(df_horizon_rmse_baseline['horizon'], df_horizon_rmse_baseline['rmse'], marker='o', linestyle='-')
        plt.title(f'Зависимость RMSE от горизонта прогноза для {model_name_baseline}')
        plt.xlabel('Горизонт прогноза (месяцы)'); plt.ylabel('RMSE')
        plt.xticks(HORIZONS_TO_EVALUATE); plt.grid(True); plt.show()

except Exception as e:
    print(f"ОШИБКА при обучении или прогнозировании {model_name_baseline}: {e}")
    aggregation_rmse[model_name_baseline] = np.nan # Записываем NaN в случае ошибки
    final_forecasts[model_name_baseline] = np.full(TEST_MONTHS, np.nan)

print(f"\n--- Раздел 2 ({model_name_baseline}) завершен ---")
print("\nТекущая таблица общих RMSE:")
print(pd.DataFrame(aggregation_rmse.items(), columns=['model', 'rmse']).sort_values(by='rmse'))
print("\nТекущая таблица RMSE по горизонтам:")
print(all_horizon_rmse.pivot(index='horizon', columns='model', values='rmse').round(4))

In [ ]:
from statsmodels.tsa.stattools import adfuller

print("\nЯчейка 5а: Анализ Стационарности Компонент (ADF Тест)")

# --- ПРЕДПОЛАГАЕМ НАЛИЧИЕ ПЕРЕМЕННЫХ ---
# train_comp_data: DataFrame с обучающими выборками компонент
# COMPONENTS_TO_MODEL: Список имен компонент
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

significance_level_adf = 0.05 # Уровень значимости для теста
adf_results = {} # Словарь для хранения результатов

print(f"Уровень значимости для ADF теста: {significance_level_adf}")

for component in COMPONENTS_TO_MODEL:
    print(f"\n--- Тест для компоненты: {component} ---")
    try:
        # Берем обучающую выборку компоненты
        component_series = train_comp_data[component].dropna()

        if component_series.empty:
            print("  Ряд пуст после удаления NaN. Пропуск теста.")
            adf_results[component] = {'p_value': np.nan, 'is_stationary': None, 'error': 'Empty series'}
            continue
        if component_series.nunique() <= 1:
             print("  Ряд содержит константу. Пропуск теста ADF (неприменим).")
             # Считаем такой ряд стационарным (d=0) для простоты
             adf_results[component] = {'p_value': 0.0, 'is_stationary': True, 'error': 'Constant series'}
             continue


        # Проводим тест ADF
        adf_test = adfuller(component_series, autolag='AIC')

        adf_stat = adf_test[0]
        p_value = adf_test[1]
        crit_values = adf_test[4]

        print(f"  ADF Statistic: {adf_stat:.4f}")
        print(f"  p-value: {p_value:.4f}")
        # print("  Критические значения:")
        # for key, value in crit_values.items():
        #     print(f"    {key}: {value:.4f}")

        # Интерпретация
        if p_value <= significance_level_adf:
            print(f"  Вывод: Ряд '{component}' СТАЦИОНАРЕН (p <= {significance_level_adf}). Рекомендуется d=0.")
            adf_results[component] = {'p_value': p_value, 'is_stationary': True}
        else:
            print(f"  Вывод: Ряд '{component}' НЕ СТАЦИОНАРЕН (p > {significance_level_adf}). auto_arima определит d.")
            adf_results[component] = {'p_value': p_value, 'is_stationary': False}

    except Exception as e:
        print(f"  ОШИБКА при выполнении ADF теста для {component}: {e}")
        adf_results[component] = {'p_value': np.nan, 'is_stationary': None, 'error': str(e)}

print("\n--- Анализ стационарности компонент завершен ---")
# Можно вывести итоговый словарь adf_results, если нужно
# print("\nРезультаты ADF тестов:")
# print(adf_results)

In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.sandbox.stats.multicomp import multipletests
import matplotlib.pyplot as plt # Добавляем импорт для графиков

print("\nЯчейка 5а: Анализ Стационарности Компонент (ADF Тест с Поправкой Холма)")

# --- ПРЕДПОЛАГАЕМ НАЛИЧИЕ ПЕРЕМЕННЫХ ---
# train_comp_data: DataFrame с обучающими выборками компонент
# COMPONENTS_TO_MODEL_FINAL: Финальный список имен компонент (из Ячейки 3а)
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

significance_level_adf = 0.05 # Уровень значимости для ОДНОГО теста
alpha_fwer = 0.05 # Уровень значимости для КОНТРОЛЯ FWER
adf_results = {}
p_values_list = []
components_tested = [] # Список компонент, для которых тест был проведен

print(f"Уровень значимости для контроля FWER (Метод Холма): {alpha_fwer}")

# --- Шаг 1: Проведение ADF тестов и сбор p-value ---
print("\n--- Проведение ADF тестов для компонент ---")
for component in COMPONENTS_TO_MODEL:
    # print(f"\n--- Тест для компоненты: {component} ---") # Убрал для краткости вывода
    try:
        component_series = train_comp_data[component].dropna()

        if component_series.empty:
            # print("  Ряд пуст. Пропуск.")
            adf_results[component] = {'p_value': np.nan, 'adf_stat': np.nan, 'is_stationary_holm': None, 'notes': 'Empty series'}
            continue
        if component_series.nunique() <= 1:
            # print("  Ряд константа. Считаем стационарным (p=0).")
            adf_results[component] = {'p_value': 0.0, 'adf_stat': np.nan, 'is_stationary_holm': True, 'notes': 'Constant series'}
            # Добавляем в списки для Холма
            p_values_list.append(0.0)
            components_tested.append(component)
            continue

        adf_test = adfuller(component_series, autolag='AIC')
        adf_stat = adf_test[0]
        p_value = adf_test[1]

        adf_results[component] = {'p_value': p_value, 'adf_stat': adf_stat} # Сохраняем исходный p-value
        # Добавляем в списки для Холма
        p_values_list.append(p_value)
        components_tested.append(component)
        # print(f"  p-value (исходный): {p_value:.4f}") # Убрал для краткости

    except Exception as e:
        print(f"  ОШИБКА при ADF тесте для {component}: {e}")
        adf_results[component] = {'p_value': np.nan, 'adf_stat': np.nan, 'is_stationary_holm': None, 'notes': str(e)}

# --- Шаг 2: Применение метода Холма ---
print(f"\n--- Применение поправки Холма (alpha={alpha_fwer}) ---")
if not p_values_list:
    print("Нет валидных p-value для применения поправки.")
else:
    try:
        reject_holm, pvals_corrected_holm, _, _ = multipletests(p_values_list,
                                                                 alpha=alpha_fwer,
                                                                 method='holm')

        # Обновляем результаты в словаре adf_results
        non_stationary_components_holm = []
        for i, component in enumerate(components_tested):
            is_stationary = reject_holm[i] # True если гипотеза о нестационарности отвергнута
            adf_results[component]['p_value_holm'] = pvals_corrected_holm[i]
            adf_results[component]['is_stationary_holm'] = is_stationary
            if not is_stationary:
                 non_stationary_components_holm.append(component)

        print(f"Поправка Холма применена к {len(p_values_list)} тестам.")
        if non_stationary_components_holm:
            print("\nКомпоненты, показавшие НЕстационарность ПОСЛЕ поправки Холма:")
            for comp in non_stationary_components_holm:
                 p_orig = adf_results[comp]['p_value']
                 p_holm = adf_results[comp]['p_value_holm']
                 print(f"  - {comp} (p_orig={p_orig:.4f}, p_holm={p_holm:.4f})")
        else:
            print("\nВсе протестированные компоненты стационарны после поправки Холма.")

    except Exception as e:
        print(f"ОШИБКА при применении multipletests (Holm): {e}")
        # Помечаем все как неопределенные
        for comp in components_tested: adf_results[comp]['is_stationary_holm'] = None


# --- Шаг 3: Визуализация НЕстационарных рядов (если есть) ---
if 'non_stationary_components_holm' in locals() and non_stationary_components_holm:
    print("\n--- Визуализация рядов, определенных как НЕстационарные ---")
    n_plots = len(non_stationary_components_holm)
    n_cols_plot = 2 # По 2 графика в ряд
    n_rows_plot = int(np.ceil(n_plots / n_cols_plot))

    plt.figure(figsize=(14, 5 * n_rows_plot))
    for i, component in enumerate(non_stationary_components_holm):
        ax = plt.subplot(n_rows_plot, n_cols_plot, i + 1)
        try:
            component_series = train_comp_data[component].dropna()
            plt.plot(component_series.index, component_series, marker='.', linestyle='-', markersize=3)
            plt.title(f"{component} (p_holm={adf_results[component]['p_value_holm']:.4f})")
            plt.ylabel("Значение")
            plt.grid(True)
        except Exception as plot_e:
             print(f"Ошибка при построении графика для {component}: {plot_e}")
             ax.set_title(f"{component} - Ошибка графика")

    plt.tight_layout()
    plt.show()
else:
     print("\nВизуализация нестационарных рядов не требуется.")


print("\n--- Анализ стационарности компонент завершен ---")
# Словарь adf_results теперь содержит 'is_stationary_holm' (True/False/None)
# Его можно использовать для определения параметра 'd' в auto_arima, если нужно

In [ ]:
print("\n--- Раздел 3: Bottom-up с ARIMA Компонентами (Простой) ---")
print("\nЯчейка 7: Обучение auto_arima для Каждой Компоненты (с выбором d)")

# --- ПРЕДПОЛАГАЕМ НАЛИЧИЕ ПЕРЕМЕННЫХ ---
# train_comp_data: DataFrame с обучающими выборками компонент
# test_comp_data: DataFrame с тестовыми выборками компонент
# COMPONENTS_TO_MODEL_FINAL: Финальный список имен компонент (из Ячейки 3а)
# TEST_MONTHS: int
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

# Список компонент, для которых ADF тест был НЕ отвергнут (или показал явный тренд)
# Обнови этот список на основе твоего последнего вывода Ячейки 5а
non_stationary_candidates = [
    'Алкогольные напитки', 'Общественное питание', 'Меха и меховые изделия',
    'Табачные изделия', 'Печатные издания', 'Бытовые услуги',
    'Услуги в системе образования', 'Услуги организаций культуры',
    'Санаторно-оздоровительные услуги', 'Медицинские услуги'
]

df_forecasts_arima_comp_list = []
arima_orders_dict = {} # Словарь для хранения ИТОГОВЫХ порядков

print(f"Обучение ARIMA и генерация прогнозов для {len(COMPONENTS_TO_MODEL)} компонент...")

for component in COMPONENTS_TO_MODEL:
    print(f"\n--- Компонента: {component} ---")
    d_param = None # По умолчанию позволяем auto_arima решать

    # Определяем параметр d на основе результатов ADF и нашего анализа
    if component not in non_stationary_candidates:
        # Если компонент прошел тест на стационарность (или мы решили, что он стационарен)
        d_param = 0
        print(f"  Устанавливаем d=0 (стационарный по тесту/анализу)")
    else:
        # Для нестационарных или спорных - позволяем auto_arima выбрать d
        print(f"  Позволяем auto_arima определить d (потенциально нестационарный)")


    try:
        y_train_comp = train_comp_data[component].dropna()
        y_test_comp = test_comp_data[component]

        if y_train_comp.empty:
            print("  Обучающий ряд пуст. Пропуск.")
            forecast = np.full(TEST_MONTHS, np.nan)
            order = None
        elif y_train_comp.nunique() <= 1:
             print("  Обучающий ряд содержит константу. Прогноз = константа.")
             forecast = np.full(TEST_MONTHS, y_train_comp.iloc[0])
             order = (0,0,0) # Условно
        else:
            # Используем auto_arima с определенным d_param (0 или None)
            arima_comp_model = auto_arima(y_train_comp,
                                          start_p=1, start_q=1, max_p=5, max_q=5,
                                          m=1, seasonal=False,
                                          d=d_param, # <--- ИСПОЛЬЗУЕМ d=0 или d=None
                                          test='adf', # Тест все равно запустится, если d=None
                                          trace=False, error_action='ignore', suppress_warnings=True,
                                          stepwise=True, information_criterion='aic', n_jobs=-1)

            order = arima_comp_model.order # Получаем ИТОГОВЫЙ порядок (p,d,q)
            print(f"  Выбрана модель: {order} {arima_comp_model.seasonal_order}")

            # Генерация прогноза
            forecast = arima_comp_model.predict(n_periods=TEST_MONTHS)

        # Сохраняем итоговый порядок
        arima_orders_dict[component] = order
        # Сохраняем прогноз
        forecast_series = pd.Series(forecast, index=y_test_comp.index[:len(forecast)], name=component)
        df_forecasts_arima_comp_list.append(forecast_series)
        print(f"  Прогноз для {component} сгенерирован.")

    except Exception as e:
        print(f"  ОШИБКА при обработке {component}: {e}")
        forecast_series = pd.Series(np.nan, index=test_comp_data.index[:TEST_MONTHS], name=component)
        df_forecasts_arima_comp_list.append(forecast_series)
        arima_orders_dict[component] = None

# Объединяем прогнозы в DataFrame
df_forecasts_arima_comp = pd.concat(df_forecasts_arima_comp_list, axis=1)

print("\n--- Прогнозы ARIMA для компонент готовы ---")
print("Итоговые порядки ARIMA (p,d,q):")
# Выведем порядки в более читаемом виде
for comp, order in arima_orders_dict.items():
    print(f"  - {comp}: {order}")
# print(arima_orders_dict) # Можно вывести и весь словарь
print("\nПервые 5 строк прогнозов:")
print(df_forecasts_arima_comp.head())
print("\nПроверка на NaN в прогнозах:")
print(df_forecasts_arima_comp.isnull().sum())

print("\n--- Ячейка 7 (для Раздела 3) завершена ---")

In [ ]:
print("\nЯчейка 8: Агрегация ARIMA Прогнозов Простыми Весами (с Учетом 'Прочего')")

# --- ПРЕДПОЛАГАЕМ НАЛИЧИЕ ПЕРЕМЕННЫХ ---
# df_forecasts_arima_comp: DataFrame [ИндексТеста x 44 Компоненты] - прогнозы ARIMA
# weights_norm: DataFrame [Год x 44 Компоненты] - нормализованные веса (сумма < 1)
# sum_weights_modelled: pd.Series [Год] - Суммарный вес 44 компонент по годам (из Ячейки 4)
# train_values: Series - Обуч. выборка агрегата
# test_values: Series - Тест. выборка агрегата
# COMPONENTS_TO_MODEL_FINAL: list - Финальный список 44 компонент
# HORIZONS_TO_EVALUATE: list
# aggregation_rmse, all_horizon_rmse, final_forecasts
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

# Проверка df_forecasts_arima_comp
if 'df_forecasts_arima_comp' not in locals() or df_forecasts_arima_comp is None:
    print("ОШИБКА: DataFrame 'df_forecasts_arima_comp' не найден. Запустите Ячейку 7.")
    raise NameError("df_forecasts_arima_comp не определен")
elif df_forecasts_arima_comp.isnull().any().any():
    print("ВНИМАНИЕ: Обнаружены NaN в прогнозах компонент ARIMA. Агрегация может быть некорректна.")
    # Можно добавить обработку NaN здесь, если требуется

# --- Метод 1: Веса последнего года обучения (с масштабированием) ---
model_name_arima_lastw = "BottomUp-ARIMA-LastW"
print(f"\n--- {model_name_arima_lastw} ---")
try:
    last_train_date_agg = train_values.index.max()
    last_train_year_agg = last_train_date_agg.year

    if last_train_year_agg in weights_norm.index and last_train_year_agg in sum_weights_modelled.index:
        # Веса ТОЛЬКО для 44 компонент
        last_train_weights_44 = weights_norm.loc[last_train_year_agg, COMPONENTS_TO_MODEL]
        # Сумма весов ТОЛЬКО для 44 компонент в этом году
        sum_weights_44_last = sum_weights_modelled.loc[last_train_year_agg]

        print(f"Используются веса за {last_train_year_agg} год (сумма = {sum_weights_44_last:.4f})")

        # Шаг А: Взвешенная сумма прогнозов 44 компонент
        forecast_44_weighted = (df_forecasts_arima_comp[COMPONENTS_TO_MODEL] * last_train_weights_44).sum(axis=1)

        # Шаг Б: Масштабирование на суммарный вес
        if sum_weights_44_last > 1e-6: # Проверка деления на ноль
             final_forecast_scaled = forecast_44_weighted / sum_weights_44_last
        else:
             print("ОШИБКА: Сумма весов равна нулю, масштабирование невозможно.")
             final_forecast_scaled = pd.Series(np.nan, index=forecast_44_weighted.index)

        final_forecasts[model_name_arima_lastw] = final_forecast_scaled.values

        # Оценка
        eval_len = min(len(final_forecast_scaled), len(test_values))
        # Проверим наличие NaN в финальном прогнозе перед оценкой
        if pd.isna(final_forecast_scaled[:eval_len]).any():
             print("ПРЕДУПРЕЖДЕНИЕ: NaN в итоговом прогнозе после масштабирования.")
             overall_rmse = np.nan
        else:
             overall_rmse = np.sqrt(mean_squared_error(test_values[:eval_len], final_forecast_scaled[:eval_len]))
        print(f"Общий RMSE: {overall_rmse:.4f}")
        aggregation_rmse[model_name_arima_lastw] = overall_rmse

        # ... (код расчета RMSE по горизонтам, аналогично предыдущему, используя final_forecast_scaled) ...
        print("RMSE по горизонтам:")
        horizon_results_list = []
        for h in HORIZONS_TO_EVALUATE:
            if h <= eval_len and not pd.isna(final_forecast_scaled[:h]).any():
                rmse_h = np.sqrt(mean_squared_error(test_values[:h], final_forecast_scaled[:h]))
                # print(f"  1-{h} мес.: {rmse_h:.4f}")
                horizon_results_list.append({'model': model_name_arima_lastw, 'horizon': h, 'rmse': rmse_h})
        if horizon_results_list:
             df_horizon = pd.DataFrame(horizon_results_list)
             all_horizon_rmse = all_horizon_rmse[all_horizon_rmse['model'] != model_name_arima_lastw]
             all_horizon_rmse = pd.concat([all_horizon_rmse, df_horizon], ignore_index=True)
        print("RMSE по горизонтам рассчитаны.")


    else:
        print(f"ОШИБКА: Веса или сумма весов за {last_train_year_agg} год не найдены!")
        aggregation_rmse[model_name_arima_lastw] = np.nan

except Exception as e:
    print(f"Ошибка при расчете {model_name_arima_lastw}: {e}")
    aggregation_rmse[model_name_arima_lastw] = np.nan


# --- Метод 2: Средние веса (с масштабированием) ---
model_name_arima_avgw = "BottomUp-ARIMA-AvgW"
print(f"\n--- {model_name_arima_avgw} ---")
try:
    # Средние веса ТОЛЬКО для 44 компонент
    avg_weights_44 = weights_norm.mean()[COMPONENTS_TO_MODEL]
    # Средняя сумма весов ТОЛЬКО для 44 компонент
    avg_sum_weights_44 = sum_weights_modelled.mean()

    print(f"Используемые средние веса (средняя сумма = {avg_sum_weights_44:.4f})")

    # Шаг А: Взвешенная сумма прогнозов 44 компонент
    forecast_44_weighted_avg = (df_forecasts_arima_comp[COMPONENTS_TO_MODEL] * avg_weights_44).sum(axis=1)

    # Шаг Б: Масштабирование на средний суммарный вес
    if avg_sum_weights_44 > 1e-6:
        final_forecast_scaled_avg = forecast_44_weighted_avg / avg_sum_weights_44
    else:
        print("ОШИБКА: Средняя сумма весов равна нулю.")
        final_forecast_scaled_avg = pd.Series(np.nan, index=forecast_44_weighted_avg.index)

    final_forecasts[model_name_arima_avgw] = final_forecast_scaled_avg.values

    # Оценка
    eval_len = min(len(final_forecast_scaled_avg), len(test_values))
    if pd.isna(final_forecast_scaled_avg[:eval_len]).any():
         print("ПРЕДУПРЕЖДЕНИЕ: NaN в итоговом прогнозе после масштабирования.")
         overall_rmse = np.nan
    else:
        overall_rmse = np.sqrt(mean_squared_error(test_values[:eval_len], final_forecast_scaled_avg[:eval_len]))
    print(f"Общий RMSE: {overall_rmse:.4f}")
    aggregation_rmse[model_name_arima_avgw] = overall_rmse

    # ... (код расчета RMSE по горизонтам, используя final_forecast_scaled_avg) ...
    print("RMSE по горизонтам:")
    horizon_results_list = []
    for h in HORIZONS_TO_EVALUATE:
        if h <= eval_len and not pd.isna(final_forecast_scaled_avg[:h]).any():
           rmse_h = np.sqrt(mean_squared_error(test_values[:h], final_forecast_scaled_avg[:h]))
           # print(f"  1-{h} мес.: {rmse_h:.4f}")
           horizon_results_list.append({'model': model_name_arima_avgw, 'horizon': h, 'rmse': rmse_h})
    if horizon_results_list:
        df_horizon = pd.DataFrame(horizon_results_list)
        all_horizon_rmse = all_horizon_rmse[all_horizon_rmse['model'] != model_name_arima_avgw]
        all_horizon_rmse = pd.concat([all_horizon_rmse, df_horizon], ignore_index=True)
    print("RMSE по горизонтам рассчитаны.")


except Exception as e:
    print(f"Ошибка при расчете {model_name_arima_avgw}: {e}")
    aggregation_rmse[model_name_arima_avgw] = np.nan

# --- Вывод обновленных таблиц ---
# ... (код для вывода таблиц RMSE как в предыдущих ячейках) ...
print("\n--- Обновленное сравнение RMSE (Общий RMSE) ---")
# ... (код вывода отсортированного aggregation_rmse) ...
if 'aggregation_rmse' in locals() and aggregation_rmse:
    valid_keys = [k for k in aggregation_rmse if pd.notna(aggregation_rmse[k])]
    if valid_keys:
        sorted_rmse = {k: aggregation_rmse[k] for k in sorted(valid_keys, key=aggregation_rmse.get)}
        for model, rmse in sorted_rmse.items(): print(f"  {model}: {rmse:.4f}")
    else: print("Нет валидных RMSE для сортировки.")
else: print("Словарь aggregation_rmse пуст.")

print("\nОбновленная таблица RMSE по горизонтам:")
# ... (код вывода сводной таблицы all_horizon_rmse) ...
if 'all_horizon_rmse' in locals() and not all_horizon_rmse.empty:
    try:
        pivot_rmse_final = all_horizon_rmse.pivot(index='horizon', columns='model', values='rmse')
        if 'sorted_rmse' in locals():
             ordered_columns_keys = list(sorted_rmse.keys())
             ordered_columns_final = [col for col in ordered_columns_keys if col in pivot_rmse_final.columns]
             remaining_cols_final = [col for col in pivot_rmse_final.columns if col not in ordered_columns_final]
             pivot_rmse_final = pivot_rmse_final[ordered_columns_final + remaining_cols_final]
        print(pivot_rmse_final.round(4))
    except Exception as e: print(f"Не удалось создать сводную таблицу: {e}")
else: print("Нет данных.")


print("\n--- Ячейка 8 (с масштабированием) завершена ---")

In [ ]:
print("\nЯчейка 9: Генерация OOF-прогнозов ARIMA Компонент")

# --- ПРЕДПОЛАГАЕМ НАЛИЧИЕ ПЕРЕМЕННЫХ ---
# train_comp_data: DataFrame с обучающими выборками компонент
# components_to_model: Список имен компонент
# arima_orders_dict: Словарь с порядками ARIMA для компонент (из Ячейки 7)
# N_OOF_SPLITS: int - Количество сплитов для OOF
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

components_to_model = COMPONENTS_TO_MODEL

# --- Инициализация DataFrame для OOF прогнозов ARIMA ---
df_oof_comp_arima = pd.DataFrame(index=train_comp_data.index,
                                columns=components_to_model,
                                dtype=float)

print(f"Генерация OOF-прогнозов ARIMA для {len(components_to_model)} компонент...")
print(f"Используется TimeSeriesSplit с {N_OOF_SPLITS} фолдами.")

# --- Цикл по компонентам ---
for i, component_name in enumerate(components_to_model):
    print(f"\n({i+1}/{len(components_to_model)}) OOF для компоненты: {component_name}")

    component_series_train = train_comp_data[component_name].rename('t').dropna()
    component_order = arima_orders_dict.get(component_name)

    if component_order is None:
        print(f"  ОШИБКА: Порядок ARIMA для {component_name} не определен. Пропуск OOF.")
        continue

    print(f"  Используется порядок: {component_order}")

    # --- Подготовка к OOF ---
    tscv_oof_arima = TimeSeriesSplit(n_splits=N_OOF_SPLITS)
    oof_predictions_component = pd.Series(index=component_series_train.index, dtype=float)

    split_count = 0
    for train_idx, val_idx in tscv_oof_arima.split(component_series_train):
        split_count += 1
        train_data_split = component_series_train.iloc[train_idx]
        val_indices = component_series_train.index[val_idx]
        n_steps_val = len(val_idx)

        # Проверка на достаточный размер обучающей выборки фолда
        min_obs_req = sum(component_order) + 5 # Примерная оценка
        if len(train_data_split) < min_obs_req:
             print(f"    Фолд {split_count}/{N_OOF_SPLITS}: Пропуск (мало данных: {len(train_data_split)} < {min_obs_req})")
             continue
        else:
             #print(f"    Фолд {split_count}/{N_OOF_SPLITS}: Обучение на {len(train_data_split)}, прогноз на {n_steps_val}")
             pass # Убрал вывод для краткости

        # --- Обучение и прогноз ARIMA на фолде ---
        try:
            # Используем statsmodels ARIMA с ФИКСИРОВАННЫМ порядком
            model_oof = StatsmodelsARIMA(train_data_split, order=component_order,
                                         enforce_stationarity=False, enforce_invertibility=False).fit()
            forecast_val = model_oof.forecast(steps=n_steps_val).values

            # Сохраняем прогноз фолда
            oof_predictions_component.loc[val_indices] = forecast_val[:len(val_indices)]

        except Exception as e:
            print(f"    ОШИБКА на фолде {split_count} для {component_name}: {e}")

    # Сохраняем OOF-прогнозы для компоненты
    df_oof_comp_arima[component_name] = oof_predictions_component
    print(f"  OOF-прогнозы ARIMA для {component_name} сгенерированы.")

print("\n--- Генерация OOF-прогнозов ARIMA завершена ---")

# --- Постобработка и проверка OOF ---
print("\nКоличество НЕ-NaN OOF-прогнозов ARIMA по компонентам:")
print(df_oof_comp_arima.notna().sum())

df_oof_comp_arima_clean = df_oof_comp_arima.dropna()

if df_oof_comp_arima_clean.empty:
     print("\nОШИБКА: После удаления NaN не осталось OOF-прогнозов ARIMA.")
     # Дальнейший код обучения мета-модели выполнять не стоит
else:
    # Выравниваем таргет для МЕТА-модели (используем train_values из Раздела 1)
    train_values_aligned_meta_arima = train_values.loc[df_oof_comp_arima_clean.index]

    print(f"\nРазмер OOF DataFrame ARIMA после удаления NaN: {df_oof_comp_arima_clean.shape}")
    print(f"Размер выровненного таргета для мета-модели: {train_values_aligned_meta_arima.shape}")
    print("\nПервые 5 строк OOF-прогнозов ARIMA:")
    print(df_oof_comp_arima_clean.head())

In [ ]:
print("\nЯчейка 7: Определение Вспомогательных Функций")

# --- 1. Функция для Feature Engineering (для мета-моделей) ---
def create_meta_features(df_input, component_names, lags, windows, feature_order_meta):
    """
    Создает признаки для мета-модели на основе прогнозов компонент.
    Использует глобальные LAGS_TO_CREATE, ROLLING_WINDOWS.

    Args:
        df_input (pd.DataFrame): DataFrame с прогнозами компонент (индекс=Дата).
        component_names (list): Список названий компонент (колонок в df_input).
        lags (list): Список создаваемых лагов.
        windows (list): Список окон для скользящих статистик.
        feature_order_meta (list): Ожидаемый порядок выходных признаков.

    Returns:
        pd.DataFrame: DataFrame с оригинальными прогнозами и новыми признаками.
    """
    df = df_input.copy()
    # Используем индекс исходного df для нового DataFrame признаков
    feature_df = pd.DataFrame(index=df.index)

    # 1. Добавляем оригинальные прогнозы как базовые фичи
    for comp in component_names:
        # Добавляем префикс _pred для ясности, что это прогнозы
        feature_df[f'{comp}_pred'] = df[comp]

    # 2. Лаги
    for lag in lags:
        for comp in component_names:
            feature_df[f'{comp}_pred_lag{lag}'] = df[comp].shift(lag)

    # 3. Скользящие статистики
    for window in windows:
        for comp in component_names:
            feature_df[f'{comp}_pred_roll_mean{window}'] = df[comp].rolling(window=window, min_periods=1).mean()
            feature_df[f'{comp}_pred_roll_std{window}'] = df[comp].rolling(window=window, min_periods=1).std()

    # 4. Разности (с лагом 1)
    for comp in component_names:
        feature_df[f'{comp}_pred_diff1'] = df[comp].diff(1)

    # 5. Календарные признаки
    try:
        # Проверяем, что индекс - DatetimeIndex
        if isinstance(df.index, pd.DatetimeIndex):
            feature_df['month_meta_sin'] = np.sin(2 * np.pi * df.index.month / 12)
            feature_df['month_meta_cos'] = np.cos(2 * np.pi * df.index.month / 12)
        else:
            print("Предупреждение: Индекс не является DatetimeIndex, календарные признаки не добавлены.")
            feature_df['month_meta_sin'] = np.nan
            feature_df['month_meta_cos'] = np.nan
    except AttributeError:
         print("Предупреждение: Не удалось извлечь месяц из индекса, календарные признаки не добавлены.")
         feature_df['month_meta_sin'] = np.nan
         feature_df['month_meta_cos'] = np.nan


    # Переупорядочиваем колонки согласно feature_order_meta
    # Сначала создадим список всех колонок, которые есть в feature_df
    available_columns = feature_df.columns.tolist()
    # Создадим итоговый список колонок, беря их из feature_order_meta, если они есть
    final_columns_order = [col for col in feature_order_meta if col in available_columns]
    # Проверим, все ли колонки из feature_order_meta нашлись
    if len(final_columns_order) != len(feature_order_meta):
        missing_cols = set(feature_order_meta) - set(available_columns)
        print(f"Предупреждение: Не все ожидаемые признаки найдены/созданы! Отсутствуют: {missing_cols}")
        # Можно либо добавить недостающие колонки с NaN, либо продолжить с тем, что есть
        # Пока продолжим с тем, что есть
    feature_df = feature_df[final_columns_order]

    #print(f"Создано/отобрано признаков для мета-модели: {feature_df.shape[1]}")
    return feature_df

# Определяем ОЖИДАЕМЫЙ порядок признаков для МЕТА-моделей
# (На основе того, что генерирует функция create_meta_features)
# Важно, чтобы он был консистентным везде, где используется эта функция!
META_FEATURE_ORDER = []
for comp in COMPONENTS_TO_MODEL: META_FEATURE_ORDER.append(f'{comp}_pred')
for lag in LAGS_TO_CREATE:
    for comp in COMPONENTS_TO_MODEL: META_FEATURE_ORDER.append(f'{comp}_pred_lag{lag}')
for window in ROLLING_WINDOWS:
    for comp in COMPONENTS_TO_MODEL: META_FEATURE_ORDER.append(f'{comp}_pred_roll_mean{window}')
    for comp in COMPONENTS_TO_MODEL: META_FEATURE_ORDER.append(f'{comp}_pred_roll_std{window}')
for comp in COMPONENTS_TO_MODEL: META_FEATURE_ORDER.append(f'{comp}_pred_diff1')
META_FEATURE_ORDER.extend(['month_meta_sin', 'month_meta_cos'])


# --- 2. Функция для Feature Engineering (для моделей компонент) ---
def calculate_features_for_step(history_series: pd.Series, feature_order_comp: list):
    """
    Рассчитывает набор признаков для прогноза СЛЕДУЮЩЕГО шага компоненты.
    Использует глобальные LAGS_TO_CREATE, ROLLING_WINDOWS.

    Args:
        history_series (pd.Series): Временной ряд значений компоненты (y) с DatetimeIndex.
                                     Имя серии должно быть 't'.
        feature_order_comp (list): Ожидаемый порядок признаков для моделей компонент.

    Returns:
        pd.Series: Строка признаков для прогнозирования y(t),
                   индексированная именами признаков, или None.
    """
    features = {}
    n = len(history_series)
    if n == 0: return None
    try:
        next_index = history_series.index[-1] + pd.DateOffset(months=1)
    except (TypeError, IndexError): return None # Ошибка индекса

    # Лаги
    for lag in LAGS_TO_CREATE:
        features[f't-{lag}'] = history_series.iloc[-lag] if n >= lag else np.nan

    # Скользящие статистики
    for window in ROLLING_WINDOWS:
        if n >= window:
            window_data = history_series.iloc[-window:]
            features[f't_mean_lag{window}'] = window_data.mean()
            features[f't_std_lag{window}'] = window_data.std()
        else:
            features[f't_mean_lag{window}'] = np.nan
            features[f't_std_lag{window}'] = np.nan

    # Календарные признаки
    try:
        next_month_num = next_index.month
        features['month_sin'] = np.sin(2 * np.pi * next_month_num / 12)
        features['month_cos'] = np.cos(2 * np.pi * next_month_num / 12)
    except AttributeError: # На случай, если индекс не DatetimeIndex
        features['month_sin'] = np.nan
        features['month_cos'] = np.nan

    features_series = pd.Series(features)
    if features_series.isnull().any(): return None
    try:
        return features_series[feature_order_comp] # Используем порядок для компонент
    except KeyError as e:
        print(f"Ошибка calculate_features_for_step: Несовпадение имен признаков! {e}")
        return None

# Определяем ОЖИДАЕМЫЙ порядок признаков для моделей КОМПОНЕНТ
# (На основе того, что генерирует функция calculate_features_for_step)
COMP_FEATURE_ORDER = FEATURE_ORDER # Используем настройку из Ячейки 2

# --- 3. Функция для рекурсивного прогноза (обновленная) ---
def recursive_predict(model, initial_history_series: pd.Series, n_steps: int,
                      feature_calculator, feature_order_func: list, scaler=None): # Добавили feature_order_func
    """
    Генерирует рекурсивный многошаговый прогноз.

    Args:
        model: Обученная ML модель (с методом predict).
        initial_history_series (pd.Series): Исходная история y до начала прогноза. Имя должно быть 't'.
        n_steps (int): Количество шагов прогноза.
        feature_calculator (function): Функция, рассчитывающая признаки для шага.
        feature_order_func (list): Порядок колонок признаков, ожидаемый функцией калькулятора и моделью.
        scaler (object, optional): Scaler для масштабирования признаков (если нужен).

    Returns:
        np.array: Массив с прогнозами на n_steps шагов.
    """
    current_history = initial_history_series.copy()
    predictions = []
    #print(f"Запуск рекурсивного прогноза для {model.__class__.__name__} на {n_steps} шагов...")
    for i in range(n_steps):
        # Рассчитываем признаки, передавая правильный порядок
        features_for_step = feature_calculator(current_history, feature_order_func)

        if features_for_step is None:
            print(f"  Предупреждение: Не удалось рассчитать признаки на шаге {i+1} для {model.__class__.__name__}. Заполняем NaN.")
            # Заполняем оставшиеся прогнозы NaN
            predictions.extend([np.nan] * (n_steps - i))
            break

        # Подготовка признаков для модели
        features_df = features_for_step.to_frame().T
        if scaler:
             # Масштабируем ПЕРЕД подачей в модель
             try:
                 features_scaled = scaler.transform(features_df)
                 model_input = features_scaled
             except Exception as scale_e:
                 print(f"  Ошибка масштабирования на шаге {i+1}: {scale_e}")
                 predictions.extend([np.nan] * (n_steps - i))
                 break
        else:
             model_input = features_df

        # Прогноз на 1 шаг
        try:
             next_pred = model.predict(model_input)[0]
        except Exception as pred_e:
             print(f"  Ошибка предсказания модели {model.__class__.__name__} на шаге {i+1}: {pred_e}")
             predictions.extend([np.nan] * (n_steps - i))
             break

        # Сохраняем прогноз
        predictions.append(next_pred)

        # Обновляем историю
        try:
            next_index = current_history.index[-1] + pd.DateOffset(months=1)
            current_history = pd.concat([current_history, pd.Series([next_pred], index=[next_index], name='t')])
        except (TypeError, IndexError, ValueError) as hist_e:
             print(f"  Ошибка обновления истории на шаге {i+1}: {hist_e}")
             # Продолжаем без обновления истории, если это возможно, но прогнозы могут ухудшиться
             # Или прерываем:
             # predictions.extend([np.nan] * (n_steps - i))
             # break
             pass # Пока просто продолжаем

    #print(f"Рекурсивный прогноз для {model.__class__.__name__} завершен.")
    return np.array(predictions)


# --- 4. Функции для обучения и прогноза моделей компонент (обновленные) ---

def train_predict_arima(y_train_comp, n_steps):
    """Обучает auto_arima (с d=0) и делает рекурсивный прогноз."""
    print(f"    Обучение ARIMA для ряда длиной {len(y_train_comp)}...")
    try:
        model = auto_arima(y_train_comp, start_p=1, start_q=1, max_p=3, max_q=3,
                           m=1, seasonal=False, d=0, test='adf', # Используем d=0
                           trace=False, error_action='ignore', suppress_warnings=True,
                           stepwise=True, information_criterion='aic', n_jobs=1)
        order = model.order
        print(f"    ARIMA обучена, порядок: {order}")
        # Используем statsmodels для рекурсивного прогноза (стабильнее)
        history_arima = list(y_train_comp)
        arima_forecast_rec = []
        for _ in range(n_steps):
            model_rec = StatsmodelsARIMA(history_arima, order=order,
                                        enforce_stationarity=False, enforce_invertibility=False).fit()
            next_pred = model_rec.forecast(steps=1)[0]
            arima_forecast_rec.append(next_pred)
            history_arima.append(next_pred)
        print("    Рекурсивный прогноз ARIMA выполнен.")
        return np.array(arima_forecast_rec), order
    except Exception as e:
        print(f"    Ошибка ARIMA: {e}")
        return np.full(n_steps, np.nan), None

def train_predict_ml(model_class, model_params_grid, y_train_comp, n_steps,
                     feature_calculator, feature_order_ml, tscv_ml):
    """Подбирает гиперпараметры, обучает ML модель (XGB, RF) и делает рекурсивный прогноз."""
    model_name = model_class.__name__.replace('Regressor','')
    print(f"    Подготовка данных и GridSearchCV для {model_name}...")
    try:
        # Подготовка признаков для обучения и CV
        data_ml_comp = pd.DataFrame({'t': y_train_comp})
        # ... (код генерации признаков data_ml_comp как в предыдущей версии) ...
        for lag in LAGS_TO_CREATE: data_ml_comp[f't-{lag}'] = data_ml_comp['t'].shift(lag)
        target_series = data_ml_comp['t']
        for window in ROLLING_WINDOWS:
            data_ml_comp[f't_mean_lag{window}'] = target_series.rolling(window=window, min_periods=1).mean().shift(1)
            data_ml_comp[f't_std_lag{window}'] = target_series.rolling(window=window, min_periods=1).std().shift(1)
        month_num = data_ml_comp.index.month
        data_ml_comp['month_sin'] = np.sin(2 * np.pi * month_num / 12)
        data_ml_comp['month_cos'] = np.cos(2 * np.pi * month_num / 12)
        data_ml_comp = data_ml_comp.dropna()

        # Внутри функции train_predict_ml, ПЕРЕД X_train_comp = ...
        print(f"    Размер data_ml_comp ПОСЛЕ dropna: {data_ml_comp.shape}")
        print(f"    Колонки data_ml_comp: {data_ml_comp.columns.tolist()}")
        print(f"    Ожидаемый порядок feature_order_ml: {feature_order_ml}")
        # Добавим проверку на пустоту еще раз
        if data_ml_comp.empty:
            print(f"    ОШИБКА: data_ml_comp пуст после dropna!")
            return np.full(n_steps, np.nan)

        # Теперь строка с возможной ошибкой:
        X_train_comp = data_ml_comp[feature_order_ml]
        y_train_comp_aligned = data_ml_comp['t']

        X_train_comp = data_ml_comp[feature_order_ml]
        y_train_comp_aligned = data_ml_comp['t']

        if X_train_comp.empty or len(X_train_comp) < tscv_ml.get_n_splits() + 1: # Проверка для CV
            print(f"    Ошибка {model_name}: Недостаточно данных ({len(X_train_comp)}) для GridSearchCV.")
            return np.full(n_steps, np.nan)

        # GridSearchCV
        base_model = model_class(random_state=RANDOM_SEED, n_jobs=-1)
        gs = GridSearchCV(estimator=base_model,
                          param_grid=model_params_grid,
                          cv=tscv_ml, # Используем переданный tscv
                          scoring='neg_root_mean_squared_error',
                          n_jobs=-1,
                          refit=True,
                          verbose=0) # verbose=0 для краткости в цикле
        gs.fit(X_train_comp, y_train_comp_aligned)
        best_model = gs.best_estimator_
        print(f"    {model_name} обучен. Лучшие параметры: {gs.best_params_}. Лучший CV RMSE: {-gs.best_score_:.4f}")

        # Рекурсивный прогноз с лучшей моделью
        print(f"    Генерация рекурсивного прогноза {model_name}...")
        forecast = recursive_predict(best_model, y_train_comp, n_steps,
                                     feature_calculator, feature_order_ml) # Передаем порядок
        print(f"    Рекурсивный прогноз {model_name} выполнен.")
        return forecast
    except Exception as e:
        print(f"    Ошибка {model_name}: {e}")
        import traceback
        traceback.print_exc() # Печатаем traceback для отладки
        return np.full(n_steps, np.nan)

def train_predict_hybrid(y_train_comp, n_steps, arima_order, xgb_params_grid,
                         feature_calculator, feature_order_hybrid, tscv_hybrid):
    """Обучает гибрид ARIMA+XGB_на_ошибках (с CV для XGB) и делает рекурсивный прогноз."""
    print(f"    Подготовка данных и GridSearchCV для Hybrid (XGB на ошибках)...")
    try:
        # 1. Обучаем ARIMA (нужен порядок)
        if arima_order is None: raise ValueError("Порядок ARIMA для гибрида не определен.")
        print(f"      Обучение базовой ARIMA {arima_order}...")
        arima_model = StatsmodelsARIMA(y_train_comp, order=arima_order,
                                       enforce_stationarity=False, enforce_invertibility=False).fit()
        arima_fitted = arima_model.fittedvalues
        first_valid_idx = arima_fitted.first_valid_index()
        if first_valid_idx is None: raise ValueError("ARIMA fit failed in hybrid")
        y_train_aligned = y_train_comp.loc[first_valid_idx:]
        arima_fitted_aligned = arima_fitted.loc[first_valid_idx:]
        arima_errors = y_train_aligned - arima_fitted_aligned
        print(f"      Базовая ARIMA обучена, ошибки рассчитаны.")

        # 2. Готовим признаки для XGBoost
        # ... (код генерации признаков data_ml_comp как в train_predict_ml) ...
        data_ml_comp = pd.DataFrame({'t': y_train_comp})
        for lag in LAGS_TO_CREATE: data_ml_comp[f't-{lag}'] = data_ml_comp['t'].shift(lag)
        target_series = data_ml_comp['t']
        for window in ROLLING_WINDOWS:
            data_ml_comp[f't_mean_lag{window}'] = target_series.rolling(window=window, min_periods=1).mean().shift(1)
            data_ml_comp[f't_std_lag{window}'] = target_series.rolling(window=window, min_periods=1).std().shift(1)
        month_num = data_ml_comp.index.month
        data_ml_comp['month_sin'] = np.sin(2 * np.pi * month_num / 12)
        data_ml_comp['month_cos'] = np.cos(2 * np.pi * month_num / 12)
        # Выравниваем признаки с ошибками
        X_train_xgb_aligned = data_ml_comp.loc[arima_errors.index, feature_order_hybrid].dropna()
        arima_errors_aligned = arima_errors.loc[X_train_xgb_aligned.index]

        if X_train_xgb_aligned.empty or len(X_train_xgb_aligned) < tscv_hybrid.get_n_splits() + 1:
             print(f"      Ошибка Гибрид: Недостаточно данных ({len(X_train_xgb_aligned)}) для GridSearchCV XGB.")
             return np.full(n_steps, np.nan)
        print(f"      Признаки для XGB на ошибках готовы ({X_train_xgb_aligned.shape}).")

        # 3. GridSearchCV для XGBoost на ошибках
        xgb_error_base = xgb.XGBRegressor(objective='reg:squarederror', random_state=RANDOM_SEED, n_jobs=-1)
        gs_xgb_err = GridSearchCV(estimator=xgb_error_base,
                                 param_grid=xgb_params_grid, # Используем сетку для XGB
                                 cv=tscv_hybrid,
                                 scoring='neg_root_mean_squared_error',
                                 n_jobs=-1, refit=True, verbose=0)
        gs_xgb_err.fit(X_train_xgb_aligned, arima_errors_aligned)
        best_xgb_error_model = gs_xgb_err.best_estimator_
        print(f"      XGB на ошибках обучен. Лучшие параметры: {gs_xgb_err.best_params_}. Лучший CV RMSE: {-gs_xgb_err.best_score_:.4f}")

        # 4. Рекурсивный прогноз
        print("      Генерация рекурсивного прогноза гибрида...")
        # Прогноз ARIMA (делаем снова, т.к. history нужна чистая)
        history_arima = list(y_train_comp)
        arima_forecast_rec = []
        for _ in range(n_steps):
            model_rec = StatsmodelsARIMA(history_arima, order=arima_order,
                                        enforce_stationarity=False, enforce_invertibility=False).fit()
            next_pred_a = model_rec.forecast(steps=1)[0]
            arima_forecast_rec.append(next_pred_a)
            history_arima.append(next_pred_a) # Добавляем прогноз ARIMA в историю для ARIMA
        arima_forecast_rec = np.array(arima_forecast_rec)
        print("        Прогноз ARIMA для гибрида готов.")

        # Рекурсивный прогноз ошибок XGBoost (использует историю реальных данных + прогнозы ошибок)
        # Важно: recursive_predict для ошибок должен использовать историю y_train_comp для расчета признаков!
        xgb_error_forecast_rec = recursive_predict(best_xgb_error_model, y_train_comp, n_steps,
                                                  feature_calculator, feature_order_hybrid)
        print("        Прогноз ошибок XGB для гибрида готов.")

        # Комбинируем
        final_forecast = arima_forecast_rec + xgb_error_forecast_rec
        print("    Рекурсивный прогноз Гибрида выполнен.")
        return final_forecast

    except Exception as e:
        print(f"    Ошибка Гибрид: {e}")
        import traceback
        traceback.print_exc()
        return np.full(n_steps, np.nan)


# --- 5. Функция для оценки моделей и выбора лучшей (ОБНОВЛЕННАЯ) ---
def evaluate_component_models_cv(y_train_comp, y_test_comp, component_name, horizons, weights_rmse,
                                 rf_param_grid, xgb_param_grid, tscv_comp):
    """Обучает все модели для компоненты (с CV для ML), оценивает и выбирает лучшую."""
    model_forecasts = {}
    model_rmses = pd.DataFrame(columns=['model', 'horizon', 'rmse'])
    best_params = {} # Сохраняем лучшие параметры

    # ARIMA
    print("  Модель: ARIMA")
    forecast_arima, order_arima = train_predict_arima(y_train_comp, len(y_test_comp))
    model_forecasts['ARIMA'] = forecast_arima
    best_params['ARIMA'] = {'order': order_arima} # Сохраняем порядок
    if order_arima is not None:
        for h in horizons:
            rmse = np.sqrt(mean_squared_error(y_test_comp[:h], forecast_arima[:h]))
            model_rmses = pd.concat([model_rmses, pd.DataFrame([{'model':'ARIMA', 'horizon':h, 'rmse':rmse}])], ignore_index=True)

    # XGBoost
    print("\n  Модель: XGBoost")
    forecast_xgb = train_predict_ml(xgb.XGBRegressor, xgb_param_grid, y_train_comp, len(y_test_comp),
                                    calculate_features_for_step, COMP_FEATURE_ORDER, tscv_comp)
    model_forecasts['XGBoost'] = forecast_xgb
    # Не сохраняем параметры XGBoost из этой функции, т.к. best_params из CV есть внутри
    for h in horizons:
        rmse = np.sqrt(mean_squared_error(y_test_comp[:h], forecast_xgb[:h]))
        model_rmses = pd.concat([model_rmses, pd.DataFrame([{'model':'XGBoost', 'horizon':h, 'rmse':rmse}])], ignore_index=True)

    # RandomForest
    print("\n  Модель: RandomForest")
    forecast_rf = train_predict_ml(RandomForestRegressor, rf_param_grid, y_train_comp, len(y_test_comp),
                                   calculate_features_for_step, COMP_FEATURE_ORDER, tscv_comp)
    model_forecasts['RandomForest'] = forecast_rf
    for h in horizons:
        rmse = np.sqrt(mean_squared_error(y_test_comp[:h], forecast_rf[:h]))
        model_rmses = pd.concat([model_rmses, pd.DataFrame([{'model':'RandomForest', 'horizon':h, 'rmse':rmse}])], ignore_index=True)

    # Гибрид ARIMA+XGB_err_Rec
    print("\n  Модель: ARIMA+XGB_err_Rec")
    if order_arima is not None: # Гибрид возможен только если ARIMA обучилась
        forecast_hybrid = train_predict_hybrid(y_train_comp, len(y_test_comp), order_arima, xgb_param_grid,
                                              calculate_features_for_step, COMP_FEATURE_ORDER, tscv_comp)
        model_forecasts['ARIMA+XGB_err_Rec'] = forecast_hybrid
        for h in horizons:
             rmse = np.sqrt(mean_squared_error(y_test_comp[:h], forecast_hybrid[:h]))
             model_rmses = pd.concat([model_rmses, pd.DataFrame([{'model':'ARIMA+XGB_err_Rec', 'horizon':h, 'rmse':rmse}])], ignore_index=True)
    else:
         print("    Пропуск Гибрида, т.к. ARIMA не обучилась.")
         model_forecasts['ARIMA+XGB_err_Rec'] = np.full(len(y_test_comp), np.nan)


    # Выбор лучшей модели
    print("\n  Расчет взвешенного RMSE для выбора лучшей модели...")
    weighted_rmses = {}
    models_evaluated = model_rmses['model'].unique()
    for model_name in models_evaluated:
        rmses = model_rmses[(model_rmses['model'] == model_name) & (model_rmses['horizon'].isin(weights_rmse.keys()))].set_index('horizon')['rmse']
        common_horizons = rmses.index.intersection(weights_rmse.keys())
        if not common_horizons.empty and not rmses.loc[common_horizons].isnull().any(): # Проверка на NaN
            weighted_avg_rmse = (rmses.loc[common_horizons] * weights_rmse.loc[common_horizons]).sum()
            weighted_rmses[model_name] = weighted_avg_rmse
            print(f"    {model_name}: Weighted RMSE = {weighted_avg_rmse:.4f}")
        else:
            print(f"    {model_name}: Недостаточно RMSE или есть NaN для расчета взвешенного среднего.")
            weighted_rmses[model_name] = np.inf # Ставим бесконечность, чтобы не выбрать

    if not weighted_rmses or all(np.isinf(v) for v in weighted_rmses.values()):
        best_model_name = "ARIMA" # Запасной вариант - ARIMA
        print(f"ПРЕДУПРЕЖДЕНИЕ: Не удалось выбрать лучшую модель для {component_name}. Выбрана ARIMA по умолчанию.")
    else:
        best_model_name = min(weighted_rmses, key=weighted_rmses.get)

    print(f"\n  Лучшая модель для компоненты '{component_name}': {best_model_name}")

    return model_forecasts, best_model_name, model_rmses, best_params # Возвращаем параметры

print("Вспомогательные функции определены.")

In [ ]:
print("\nЯчейка 10: Создание Признаков для Мета-моделей на ARIMA OOF")

# --- ПРЕДПОЛАГАЕМ НАЛИЧИЕ ПЕРЕМЕННЫХ ---
# df_oof_comp_arima_clean: DataFrame с OOF ARIMA прогнозами (чистый)
# train_values_aligned_meta_arima: Series с таргетом (агрегат), выровненным с OOF ARIMA
# df_forecasts_arima_comp: DataFrame с ARIMA прогнозами компонент на тесте
# components_to_model: list
# create_meta_features: function (должна быть определена в Ячейке 7)
# META_FEATURE_ORDER: list - порядок признаков для мета-моделей (определен в Ячейке 7)
# LAGS_TO_CREATE, ROLLING_WINDOWS: list (определены в Ячейке 2 или 7)
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

# --- 1. Создание признаков для ОБУЧАЮЩЕЙ выборки ---
print("Создание признаков для ОБУЧАЮЩЕЙ выборки (на ARIMA OOF)...")
try:
    # Применяем функцию к OOF прогнозам ARIMA
    X_train_meta_arima_eng = create_meta_features(
        df_input=df_oof_comp_arima_clean,
        component_names=COMPONENTS_TO_MODEL,
        lags=LAGS_TO_CREATE,
        windows=ROLLING_WINDOWS,
        feature_order_meta=META_FEATURE_ORDER # Передаем ожидаемый порядок
    )
    # Обработка NaN после создания признаков
    X_train_meta_arima_eng_clean = X_train_meta_arima_eng.dropna()

    # Выравниваем таргет еще раз под очищенные признаки
    y_train_meta_arima_aligned = train_values_aligned_meta_arima.loc[X_train_meta_arima_eng_clean.index]

    print(f"Размер обучающей выборки с признаками (ARIMA OOF): {X_train_meta_arima_eng_clean.shape}")
    print(f"Размер выровненного таргета: {y_train_meta_arima_aligned.shape}")

except Exception as e:
    print(f"Ошибка при создании обучающих признаков для мета-модели: {e}")
    # Установим переменные в None, чтобы предотвратить ошибки далее
    X_train_meta_arima_eng_clean = None
    y_train_meta_arima_aligned = None

# --- 2. Создание признаков для ТЕСТОВОЙ выборки ---
# Выполняем только если обучающие признаки созданы успешно
if X_train_meta_arima_eng_clean is not None:
    print("\nСоздание признаков для ТЕСТОВОЙ выборки (на ARIMA прогнозах теста)...")
    try:
        # Определяем, сколько истории нужно из OOF
        max_lookback_arima = max(max(LAGS_TO_CREATE, default=0), max(ROLLING_WINDOWS, default=0))

        # Проверяем, достаточно ли OOF данных для истории
        if len(df_oof_comp_arima_clean) >= max_lookback_arima:
            history_for_test_arima = df_oof_comp_arima_clean.iloc[-max_lookback_arima:]
        else:
            # Если OOF не хватает, берем всю доступную OOF историю
            print(f"Предупреждение: Недостаточно OOF истории ({len(df_oof_comp_arima_clean)}) для lookback ({max_lookback_arima}). Используется вся доступная OOF история.")
            history_for_test_arima = df_oof_comp_arima_clean.copy()

        # Объединяем историю OOF и прогнозы ARIMA на тесте
        # Важно: индексы должны быть совместимы (DatetimeIndex)
        df_for_test_features_arima = pd.concat([history_for_test_arima, df_forecasts_arima_comp])

        # Генерируем признаки
        X_test_meta_arima_eng_full = create_meta_features(
            df_input=df_for_test_features_arima,
            component_names=COMPONENTS_TO_MODEL,
            lags=LAGS_TO_CREATE,
            windows=ROLLING_WINDOWS,
            feature_order_meta=META_FEATURE_ORDER # Передаем ожидаемый порядок
        )
        # Оставляем только тестовый период
        X_test_meta_arima_eng = X_test_meta_arima_eng_full.loc[df_forecasts_arima_comp.index]

        # Проверка и обработка NaN в тесте
        if X_test_meta_arima_eng.isnull().any().any():
            print("ВНИМАНИЕ: Обнаружены NaN в тестовых признаках (ARIMA OOF). Заполняем медианой трейна...")
            for col in X_test_meta_arima_eng.columns:
                if X_test_meta_arima_eng[col].isnull().any():
                    median_val = X_train_meta_arima_eng_clean[col].median()
                    X_test_meta_arima_eng[col] = X_test_meta_arima_eng[col].fillna(median_val)
            print(f"Кол-во NaN после заполнения: {X_test_meta_arima_eng.isnull().sum().sum()}")
        else:
            print("NaN в тестовых признаках (ARIMA OOF) не обнаружены.")

        print("\n--- Признаки для мета-моделей на ARIMA OOF готовы ---")
        print("Созданы: X_train_meta_arima_eng_clean, y_train_meta_arima_aligned, X_test_meta_arima_eng")
        # Выведем размерности для контроля
        print(f"Размер X_train_meta_arima_eng_clean: {X_train_meta_arima_eng_clean.shape}")
        print(f"Размер y_train_meta_arima_aligned: {y_train_meta_arima_aligned.shape}")
        print(f"Размер X_test_meta_arima_eng: {X_test_meta_arima_eng.shape}")


    except Exception as e:
        print(f"Ошибка при создании тестовых признаков для мета-модели: {e}")
        X_test_meta_arima_eng = None # Устанавливаем в None в случае ошибки
else:
     print("Пропуск создания тестовых признаков из-за ошибки на этапе создания обучающих.")
     X_test_meta_arima_eng = None

print("\n--- Ячейка 10 завершена ---")

print("Колонки X_train_meta_arima_eng:", X_train_meta_arima_eng.columns.tolist())
print("Количество колонок:", len(X_train_meta_arima_eng.columns))
print("Размер до dropna:", X_train_meta_arima_eng.shape)
# Сравни этот вывод с тем, что было в старом ноутбуке (должно быть 35 признаков).

In [ ]:
# print("\nЯчейка 11: Обучение и Оценка ВСЕХ Мета-моделей на ARIMA OOF (с Hyperopt)") # Изменено

# --- Импорты для Hyperopt ---
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
from sklearn.preprocessing import StandardScaler # Используем StandardScaler для линейных
from sklearn.linear_model import Ridge, Lasso # RidgeCV и LassoCV убраны
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
# Убедимся, что TimeSeriesSplit определен
# from sklearn.model_selection import TimeSeriesSplit


# --- ПРЕДПОЛАГАЕМ НАЛИЧИЕ ПЕРЕМЕННЫХ ---
# X_train_meta_arima_eng_clean: DataFrame с признаками из OOF ARIMA (трейн)
# y_train_meta_arima_aligned: Series с таргетом (агрегат ИПЦ на трейне, выровненный с OOF)
# X_test_meta_arima_eng: DataFrame с признаками из прогнозов ARIMA (тест)
# test_values: Series с фактом агрегированного ИПЦ на тесте (для оценки)
# N_CV_SPLITS: int (для TimeSeriesSplit внутри objective)
# HORIZONS_TO_EVALUATE: list
# aggregation_rmse, all_horizon_rmse, final_forecasts - глобальные структуры для результатов
# RANDOM_SEED
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

# Переменные для хранения прогнозов этой ячейки
meta_forecasts_on_arima = {}
MAX_EVALS_META = 50 # Общее количество итераций для каждой мета-модели (можно настроить)
LASSO_MAX_ITER = 10000

# --- Масштабирование Признаков (для Ridge/Lasso) ---
print("Масштабирование мета-признаков для Ridge/Lasso...")
scaler_meta_arima = StandardScaler() 
X_train_meta_arima_scaled = scaler_meta_arima.fit_transform(X_train_meta_arima_eng_clean)
X_test_meta_arima_scaled = scaler_meta_arima.transform(X_test_meta_arima_eng)
print("Масштабирование выполнено.")

# Инициализируем TS Splitter для CV мета-моделей
# Важно: tscv_meta должен быть определен и соответствовать данным X_train_meta_arima_eng_clean
# Если N_CV_SPLITS - это просто число, то инициализируем здесь
tscv_meta = TimeSeriesSplit(n_splits=N_CV_SPLITS)


# --- Общая функция Objective для Hyperopt (для всех мета-моделей) ---
def objective_meta_model(params, model_type, X_train_data, y_train_data, cv_splitter, is_scaled_data=False):
    """
    Универсальная целевая функция для hyperopt для мета-моделей.
    model_type: 'ridge', 'lasso', 'rf', 'xgb'
    X_train_data: обучающие признаки (могут быть масштабированными или нет)
    is_scaled_data: флаг, указывающий, переданы ли уже масштабированные данные
    """
    model = None
    
    if model_type == 'ridge':
        model = Ridge(alpha=params['alpha'])
    elif model_type == 'lasso':
        params_copy = params.copy() # Копируем, чтобы не менять оригинальный params
        # Lasso может требовать другие параметры из space, не только alpha
        model = Lasso(alpha=params_copy.pop('alpha', 0.01), # значение по умолчанию, если alpha не в params
                      max_iter=params_copy.pop('max_iter', 10000), 
                      tol=params_copy.pop('tol', 0.001),
                      random_state=RANDOM_SEED, **params_copy)
    elif model_type == 'rf':
        # Преобразование параметров для RF
        rf_params = {k.replace('meta_rf_', ''): v for k, v in params.items()}
        rf_params['n_estimators'] = int(rf_params['n_estimators'])
        rf_params['min_samples_split'] = int(rf_params['min_samples_split'])
        rf_params['min_samples_leaf'] = int(rf_params['min_samples_leaf'])
        # max_depth и max_features остаются как есть (могут быть None или строкой)
        model = RandomForestRegressor(**rf_params, random_state=RANDOM_SEED, n_jobs=-1)
    elif model_type == 'xgb':
        xgb_params = {k.replace('meta_xgb_', ''): v for k, v in params.items()}
        xgb_params['n_estimators'] = int(xgb_params['n_estimators'])
        xgb_params['max_depth'] = int(xgb_params['max_depth'])
        model = xgb.XGBRegressor(objective='reg:squarederror', random_state=RANDOM_SEED, n_jobs=-1, **xgb_params)
    else:
        raise ValueError("Неизвестный тип модели")

    rmses = []
    try:
        # X_train_data может быть DataFrame или NumPy array
        # y_train_data должна быть Series или NumPy array
        X_data_for_split = X_train_data
        if isinstance(X_train_data, pd.DataFrame):
            X_iloc_available = True
        else: # NumPy array
            X_iloc_available = False
            
        y_data_for_split = y_train_data
        if isinstance(y_train_data, pd.Series):
            y_iloc_available = True
        else: # NumPy array
            y_iloc_available = False


        for train_idx, val_idx in cv_splitter.split(X_data_for_split):
            if X_iloc_available:
                X_fold_train, X_fold_val = X_data_for_split.iloc[train_idx], X_data_for_split.iloc[val_idx]
            else:
                X_fold_train, X_fold_val = X_data_for_split[train_idx], X_data_for_split[val_idx]
            
            if y_iloc_available:
                y_fold_train, y_fold_val = y_data_for_split.iloc[train_idx], y_data_for_split.iloc[val_idx]
            else:
                y_fold_train, y_fold_val = y_data_for_split[train_idx], y_data_for_split[val_idx]

            model.fit(X_fold_train, y_fold_train)
            preds = model.predict(X_fold_val)

            if not np.all(np.isfinite(preds)) or not np.all(np.isfinite(y_fold_val if isinstance(y_fold_val, np.ndarray) else y_fold_val.values)):
                continue
            if len(y_fold_val) == 0:
                continue
            rmse = np.sqrt(mean_squared_error(y_fold_val, preds))
            rmses.append(rmse)
        
        if not rmses: avg_rmse = 1e10
        else: avg_rmse = np.mean(rmses)
    except Exception as e:
        # print(f"Error in objective for {model_type} with params {params}: {e}") # Отладка
        avg_rmse = 1e10
    
    if not np.isfinite(avg_rmse): avg_rmse = 1e10
    return {'loss': avg_rmse, 'status': STATUS_OK}

# --- Настройка и запуск Hyperopt для каждой мета-модели ---

meta_models_config = {
    "Meta-Ridge_on_ARIMA": {
        "model_type": "ridge",
        "space": {'alpha': hp.loguniform('meta_ridge_alpha', np.log(1e-4), np.log(1e3))},
        "X_train": X_train_meta_arima_scaled, # Используем масштабированные
        "X_test": X_test_meta_arima_scaled,
        "is_scaled": True
    },
    "Meta-Lasso_on_ARIMA": {
        "model_type": "lasso",
        "space": {
            'alpha': hp.loguniform('meta_lasso_alpha', np.log(1e-5), np.log(1.0)),
            'max_iter': LASSO_MAX_ITER, # Фиксированный параметр
            'tol': 0.001                 # Фиксированный параметр
            },
        "X_train": X_train_meta_arima_scaled, # Используем масштабированные
        "X_test": X_test_meta_arima_scaled,
        "is_scaled": True
    },
    "Meta-RF_on_ARIMA": {
        "model_type": "rf",
        "space": {
            'meta_rf_n_estimators': hp.quniform('meta_rf_n_estimators', 50, 200, 25),
            'meta_rf_max_depth': hp.choice('meta_rf_max_depth', [3, 5, 7, 10, None]),
            'meta_rf_max_features': hp.choice('meta_rf_max_features', ['sqrt', 0.5, 0.7, 0.9]),
            'meta_rf_min_samples_split': hp.quniform('meta_rf_min_samples_split', 2, 10, 1),
            'meta_rf_min_samples_leaf': hp.quniform('meta_rf_min_samples_leaf', 1, 8, 1)
        },
        "X_train": X_train_meta_arima_eng_clean, # НЕмасштабированные
        "X_test": X_test_meta_arima_eng,
        "is_scaled": False
    },
    "Meta-XGB_on_ARIMA": {
        "model_type": "xgb",
        "space": {
            'meta_xgb_n_estimators': hp.quniform('meta_xgb_n_estimators', 50, 250, 25),
            'meta_xgb_learning_rate': hp.loguniform('meta_xgb_learning_rate', np.log(0.01), np.log(0.2)),
            'meta_xgb_max_depth': hp.quniform('meta_xgb_max_depth', 1, 5, 1),
            'meta_xgb_subsample': hp.uniform('meta_xgb_subsample', 0.6, 1.0),
            'meta_xgb_colsample_bytree': hp.uniform('meta_xgb_colsample_bytree', 0.6, 1.0),
            'meta_xgb_gamma': hp.uniform('meta_xgb_gamma', 0.0, 0.5),
            # Можно добавить reg_alpha и reg_lambda, если нужно
            # 'meta_xgb_reg_alpha': hp.loguniform('meta_xgb_reg_alpha', np.log(0.001), np.log(1.0)),
            # 'meta_xgb_reg_lambda': hp.loguniform('meta_xgb_reg_lambda', np.log(0.1), np.log(10.0))
        },
        "X_train": X_train_meta_arima_eng_clean, # НЕмасштабированные
        "X_test": X_test_meta_arima_eng,
        "is_scaled": False
    }
}

# Цикл по конфигурациям мета-моделей
for model_name, config in meta_models_config.items():
    print(f"\n--- Обучение и оценка с Hyperopt: {model_name} ---")
    
    trials = Trials()
    try:
        rstate_meta = np.random.default_rng(RANDOM_SEED)
    except AttributeError:
        rstate_meta = np.random.RandomState(RANDOM_SEED)

    # Адаптируем objective для передачи нужных X_train и y_train
    # y_train_meta_arima_aligned передается для всех мета-моделей
    objective_fn_with_data = lambda params: objective_meta_model(
        params, 
        config["model_type"], 
        config["X_train"], 
        y_train_meta_arima_aligned, # Таргет для всех мета-моделей один
        tscv_meta, 
        config["is_scaled"]
    )

    best_params_raw = fmin(
        fn=objective_fn_with_data,
        space=config["space"],
        algo=tpe.suggest,
        max_evals=MAX_EVALS_META,
        trials=trials,
        rstate=rstate_meta,
        show_progressbar=True
    )
    
    best_final_params = space_eval(config["space"], best_params_raw)
    # Приведение типов для финальной модели
    if config["model_type"] == 'rf' or config["model_type"] == 'xgb':
        best_final_params[f'meta_{config["model_type"]}_n_estimators'] = int(best_final_params[f'meta_{config["model_type"]}_n_estimators'])
        if f'meta_{config["model_type"]}_max_depth' in best_final_params and best_final_params[f'meta_{config["model_type"]}_max_depth'] is not None :
             best_final_params[f'meta_{config["model_type"]}_max_depth'] = int(best_final_params[f'meta_{config["model_type"]}_max_depth'])
        if config["model_type"] == 'rf':
            best_final_params[f'meta_rf_min_samples_split'] = int(best_final_params[f'meta_rf_min_samples_split'])
            best_final_params[f'meta_rf_min_samples_leaf'] = int(best_final_params[f'meta_rf_min_samples_leaf'])


    best_cv_rmse = trials.best_trial['result']['loss']
    print(f"  Лучшие найденные параметры: {best_final_params}")
    print(f"  Лучший RMSE на кросс-валидации: {best_cv_rmse:.4f}")

    # Обучение финальной мета-модели
    final_model = None
    clean_params_for_fit = {k.replace(f'meta_{config["model_type"]}_', ''): v for k,v in best_final_params.items()}

    if config["model_type"] == 'ridge':
        final_model = Ridge(**clean_params_for_fit)
        final_model.fit(config["X_train"], y_train_meta_arima_aligned)
    elif config["model_type"] == 'lasso':
        # Убедимся, что max_iter и tol передаются, если они не в space
        lasso_fit_params = clean_params_for_fit.copy()
        lasso_fit_params.setdefault('max_iter', LASSO_MAX_ITER)
        lasso_fit_params.setdefault('tol', 0.001)
        final_model = Lasso(random_state=RANDOM_SEED, **lasso_fit_params)
        final_model.fit(config["X_train"], y_train_meta_arima_aligned)
        n_features_selected = np.sum(final_model.coef_ != 0)
        print(f"  Количество отобранных признаков: {n_features_selected} из {config['X_train'].shape[1] if isinstance(config['X_train'], pd.DataFrame) else config['X_train'].shape[1]}")
    elif config["model_type"] == 'rf':
        final_model = RandomForestRegressor(**clean_params_for_fit, random_state=RANDOM_SEED, n_jobs=-1)
        final_model.fit(config["X_train"], y_train_meta_arima_aligned)
    elif config["model_type"] == 'xgb':
        final_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=RANDOM_SEED, n_jobs=-1, **clean_params_for_fit)
        final_model.fit(config["X_train"], y_train_meta_arima_aligned)

    # Прогноз и Оценка
    if final_model:
        y_pred = final_model.predict(config["X_test"])
        meta_forecasts_on_arima[model_name] = y_pred
        final_forecasts[model_name] = y_pred # Сохраняем в общий словарь

        overall_rmse = np.sqrt(mean_squared_error(test_values, y_pred))
        print(f"  Общий RMSE: {overall_rmse:.4f}")
        aggregation_rmse[model_name] = overall_rmse

        print("  RMSE по горизонтам:")
        horizon_results_list = []
        for h_eval in HORIZONS_TO_EVALUATE: # Переименовал h -> h_eval
            rmse_h_val = np.sqrt(mean_squared_error(test_values[:h_eval], y_pred[:h_eval])) # Переименовал rmse_h -> rmse_h_val
            print(f"    1-{h_eval} мес.: {rmse_h_val:.4f}")
            horizon_results_list.append({'model': model_name, 'horizon': h_eval, 'rmse': rmse_h_val})
        df_horizon = pd.DataFrame(horizon_results_list)
        all_horizon_rmse = all_horizon_rmse[all_horizon_rmse['model'] != model_name]
        all_horizon_rmse = pd.concat([all_horizon_rmse, df_horizon], ignore_index=True)
    else:
        print(f"  Не удалось обучить финальную модель {model_name}")
        aggregation_rmse[model_name] = np.nan


# --- Вывод обновленных таблиц ---
# ... (твой код для вывода aggregation_rmse и all_horizon_rmse) ...

print("\n--- Ячейка 11 (Hyperopt Meta-models) завершена ---")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA as StatsmodelsARIMA
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import traceback # Уже используется

# НОВЫЕ ИМПОРТЫ для HYPEROPT
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll.base import scope # Для целочисленных параметров hp.quniform
import hyperopt

# --- ПРЕДПОЛАГАЕМЫЕ ГЛОБАЛЬНЫЕ КОНСТАНТЫ (определите их значения) ---
# COMPONENTS_TO_MODEL = ['C1', 'C2'] # Пример
# LAGS_TO_CREATE = [1, 2, 3, 6]
# ROLLING_WINDOWS = [3, 6]
# COMP_FEATURE_ORDER = [...] # Определите на основе Ячейки 2
# META_FEATURE_ORDER = [...] # Определите на основе Ячейки 7
# RANDOM_SEED = 42
# N_CV_SPLITS = 3
# MAX_EVALS_HYPEROPT = 50 # Количество итераций для Hyperopt
# HORIZONS_TO_EVALUATE = [1, 3, 6, 12]
# TEST_MONTHS = 12
# WEIGHTS_NORMALIZED_RMSE = pd.Series(...) # Определите веса
# tscv_component_cv = TimeSeriesSplit(n_splits=N_CV_SPLITS)

tscv_component_cv = TimeSeriesSplit(n_splits=N_CV_SPLITS)
MAX_EVALS_HYPEROPT = 50 

# --- Ячейка 7: Определение Вспомогательных Функций (Обновлено для Hyperopt) ---
print("\nЯчейка 7: Определение Вспомогательных Функций (Обновлено для Hyperopt)")

# --- 1. Функция для Feature Engineering (для мета-моделей) ---
# Оставляем без изменений, так как она не зависит от GridSearchCV/Hyperopt
def create_meta_features(df_input, component_names, lags, windows, feature_order_meta):
    df = df_input.copy()
    feature_df = pd.DataFrame(index=df.index)
    for comp in component_names:
        feature_df[f'{comp}_pred'] = df[comp]
    for lag in lags:
        for comp in component_names:
            feature_df[f'{comp}_pred_lag{lag}'] = df[comp].shift(lag)
    for window in windows:
        for comp in component_names:
            feature_df[f'{comp}_pred_roll_mean{window}'] = df[comp].rolling(window=window, min_periods=1).mean()
            feature_df[f'{comp}_pred_roll_std{window}'] = df[comp].rolling(window=window, min_periods=1).std()
    for comp in component_names:
        feature_df[f'{comp}_pred_diff1'] = df[comp].diff(1)
    try:
        if isinstance(df.index, pd.DatetimeIndex):
            feature_df['month_meta_sin'] = np.sin(2 * np.pi * df.index.month / 12)
            feature_df['month_meta_cos'] = np.cos(2 * np.pi * df.index.month / 12)
        else:
            print("Предупреждение: Индекс не является DatetimeIndex, календарные признаки не добавлены.")
            feature_df['month_meta_sin'] = np.nan
            feature_df['month_meta_cos'] = np.nan
    except AttributeError:
         print("Предупреждение: Не удалось извлечь месяц из индекса, календарные признаки не добавлены.")
         feature_df['month_meta_sin'] = np.nan
         feature_df['month_meta_cos'] = np.nan

    available_columns = feature_df.columns.tolist()
    final_columns_order = [col for col in feature_order_meta if col in available_columns]
    if len(final_columns_order) != len(feature_order_meta):
        missing_cols = set(feature_order_meta) - set(available_columns)
        print(f"Предупреждение: Не все ожидаемые признаки найдены/созданы! Отсутствуют: {missing_cols}")
    feature_df = feature_df[final_columns_order]
    return feature_df

# --- 2. Функция для Feature Engineering (для моделей компонент) ---
# Оставляем без изменений
def calculate_features_for_step(history_series: pd.Series, feature_order_comp: list):
    features = {}
    n = len(history_series)
    if n == 0: return None
    try:
        if not isinstance(history_series.index, pd.DatetimeIndex):
            # print("Предупреждение: history_series.index не является DatetimeIndex в calculate_features_for_step.")
            return None # Или попытаться преобразовать, но безопаснее вернуть None
        next_index = history_series.index[-1] + pd.DateOffset(months=1)
    except (TypeError, IndexError): return None

    for lag in LAGS_TO_CREATE:
        features[f't-{lag}'] = history_series.iloc[-lag] if n >= lag else np.nan
    for window in ROLLING_WINDOWS:
        if n >= window:
            window_data = history_series.iloc[-window:]
            features[f't_mean_lag{window}'] = window_data.mean()
            features[f't_std_lag{window}'] = window_data.std()
        else:
            features[f't_mean_lag{window}'] = np.nan
            features[f't_std_lag{window}'] = np.nan
    try:
        next_month_num = next_index.month
        features['month_sin'] = np.sin(2 * np.pi * next_month_num / 12)
        features['month_cos'] = np.cos(2 * np.pi * next_month_num / 12)
    except AttributeError:
        features['month_sin'] = np.nan
        features['month_cos'] = np.nan
    features_series = pd.Series(features)
    try:
        return features_series.reindex(feature_order_comp)
    except KeyError as e:
        print(f"Ошибка calculate_features_for_step: Несовпадение имен признаков! {e}")
        return None

# --- 3. Функция для рекурсивного прогноза ---
# Оставляем без изменений
def recursive_predict(model, initial_history_series: pd.Series, n_steps: int,
                      feature_calculator, feature_order_func: list, scaler=None):
    current_history = initial_history_series.copy()
    predictions = []
    for i in range(n_steps):
        features_for_step = feature_calculator(current_history, feature_order_func)
        if features_for_step is None or features_for_step.isnull().any():
            # print(f"  Предупреждение: Не удалось рассчитать признаки или есть NaN на шаге {i+1} для {model.__class__.__name__}. Заполняем NaN.")
            predictions.extend([np.nan] * (n_steps - i))
            break
        features_df = features_for_step.to_frame().T
        if scaler:
             try:
                 features_scaled = scaler.transform(features_df)
                 model_input = features_scaled
             except Exception as scale_e:
                 print(f"  Ошибка масштабирования на шаге {i+1}: {scale_e}")
                 predictions.extend([np.nan] * (n_steps - i))
                 break
        else:
             model_input = features_df
        try:
             next_pred = model.predict(model_input)[0]
        except Exception as pred_e:
             print(f"  Ошибка предсказания модели {model.__class__.__name__} на шаге {i+1}: {pred_e}")
             predictions.extend([np.nan] * (n_steps - i))
             break
        predictions.append(next_pred)
        try:
            if not isinstance(current_history.index, pd.DatetimeIndex):
                # print("Предупреждение: current_history.index не является DatetimeIndex в recursive_predict.")
                # Попытка преобразования или прерывание
                current_history.index = pd.to_datetime(current_history.index, errors='coerce')
                if current_history.index.isnull().any():
                     raise ValueError("Ошибка преобразования индекса истории в DatetimeIndex во время рекурсии.")

            next_idx_val = current_history.index[-1] + pd.DateOffset(months=1)
            new_data_point = pd.Series([next_pred], index=[next_idx_val], name='t')
            current_history = pd.concat([current_history, new_data_point])
        except (TypeError, IndexError, ValueError) as hist_e:
             print(f"  Ошибка обновления истории на шаге {i+1}: {hist_e}")
             predictions.extend([np.nan] * (n_steps - i))
             break
    return np.array(predictions)

# --- 4. Функции для обучения и прогноза моделей компонент (обновленные для Hyperopt) ---

def train_predict_arima(y_train_comp, n_steps):
    """Обучает auto_arima (с d=0) и делает рекурсивный прогноз."""
    print(f"    Обучение ARIMA для ряда длиной {len(y_train_comp)}...")
    try:
        model = auto_arima(y_train_comp, start_p=1, start_q=1, max_p=5, max_q=5,
                           m=1, seasonal=False, d=0, test='adf',
                           trace=False, error_action='ignore', suppress_warnings=True,
                           stepwise=True, information_criterion='aic', n_jobs=1)
        order = model.order
        print(f"    ARIMA обучена, порядок: {order}")
        history_arima = list(y_train_comp)
        arima_forecast_rec = []
        for _ in range(n_steps):
            model_rec = StatsmodelsARIMA(history_arima, order=order,
                                        enforce_stationarity=False, enforce_invertibility=False).fit()
            next_pred = model_rec.forecast(steps=1)[0]
            arima_forecast_rec.append(next_pred)
            history_arima.append(next_pred)
        print("    Рекурсивный прогноз ARIMA выполнен.")
        return np.array(arima_forecast_rec), order, {'order': order} # Возвращаем параметры
    except Exception as e:
        print(f"    Ошибка ARIMA: {e}")
        return np.full(n_steps, np.nan), None, {}

def train_predict_ml(model_class, hyperopt_space, y_train_comp, n_steps,
                     feature_calculator, feature_order_ml, tscv_ml, max_evals_hyperopt,
                     component_name_log="ML_model"): # Добавлен параметр для логгирования
    """Подбирает гиперпараметры с Hyperopt, обучает ML модель и делает рекурсивный прогноз."""
    model_name = model_class.__name__.replace('Regressor','')
    print(f"    [{component_name_log}] Подготовка данных и Hyperopt для {model_name}...")

    try:
        data_ml_comp = pd.DataFrame({'t': y_train_comp})
        for lag in LAGS_TO_CREATE: data_ml_comp[f't-{lag}'] = data_ml_comp['t'].shift(lag)
        target_series = data_ml_comp['t']
        for window in ROLLING_WINDOWS:
            data_ml_comp[f't_mean_lag{window}'] = target_series.rolling(window=window, min_periods=1).mean().shift(1)
            data_ml_comp[f't_std_lag{window}'] = target_series.rolling(window=window, min_periods=1).std().shift(1)
        if isinstance(data_ml_comp.index, pd.DatetimeIndex):
            month_num = data_ml_comp.index.month
            data_ml_comp['month_sin'] = np.sin(2 * np.pi * month_num / 12)
            data_ml_comp['month_cos'] = np.cos(2 * np.pi * month_num / 12)
        else:
            print(f"    Предупреждение [{component_name_log} - {model_name}]: Индекс не DatetimeIndex. Календарные признаки будут NaN.")
            data_ml_comp['month_sin'] = np.nan
            data_ml_comp['month_cos'] = np.nan
        data_ml_comp = data_ml_comp.dropna()

        if data_ml_comp.empty:
            print(f"    ОШИБКА [{component_name_log} - {model_name}]: data_ml_comp пуст после dropna!")
            return np.full(n_steps, np.nan), {}

        X_train_comp = data_ml_comp[feature_order_ml]
        y_train_comp_aligned = data_ml_comp['t']

        min_samples_for_cv = 0
        for train_idx, _ in tscv_ml.split(X_train_comp): # Подсчет минимального размера трейн выборки в CV
            if len(train_idx) > min_samples_for_cv:
                 min_samples_for_cv = len(train_idx)
        if X_train_comp.empty or len(X_train_comp) < tscv_ml.get_n_splits() + min_samples_for_cv : # Условие для CV
            print(f"    Ошибка [{component_name_log} - {model_name}]: Недостаточно данных ({len(X_train_comp)}) для Hyperopt CV (нужно хотя бы {tscv_ml.get_n_splits() + min_samples_for_cv}).")
            return np.full(n_steps, np.nan), {}

        # --- Hyperopt Objective Function ---
        def objective(params):
            # Преобразование типов для параметров, если hyperopt возвращает float для int
            current_params = params.copy() # Работаем с копией
            if 'n_estimators' in current_params: current_params['n_estimators'] = int(current_params['n_estimators'])
            if 'max_depth' in current_params and current_params['max_depth'] is not None : current_params['max_depth'] = int(current_params['max_depth'])
            if 'min_samples_leaf' in current_params: current_params['min_samples_leaf'] = int(current_params['min_samples_leaf'])
            if 'min_samples_split' in current_params: current_params['min_samples_split'] = int(current_params['min_samples_split'])
            # Для XGBoost специфичные параметры, например, num_leaves, если используется
            if 'num_leaves' in current_params: current_params['num_leaves'] = int(current_params['num_leaves'])


            model = model_class(**current_params, random_state=RANDOM_SEED, n_jobs=-1)
            cv_scores = []
            for train_idx, val_idx in tscv_ml.split(X_train_comp):
                X_cv_train, X_cv_val = X_train_comp.iloc[train_idx], X_train_comp.iloc[val_idx]
                y_cv_train, y_cv_val = y_train_comp_aligned.iloc[train_idx], y_train_comp_aligned.iloc[val_idx]
                if X_cv_train.empty or X_cv_val.empty or len(X_cv_train) < 2 or len(X_cv_val) < 1: # Добавил проверку на минимальный размер
                    continue
                try:
                    model.fit(X_cv_train, y_cv_train)
                    preds = model.predict(X_cv_val)
                    rmse = np.sqrt(mean_squared_error(y_cv_val, preds))
                    cv_scores.append(rmse)
                except Exception as cv_e:
                    # print(f"      Предупреждение CV для {model_name} с параметрами {current_params}: {cv_e}")
                    cv_scores.append(np.inf) # Штрафуем за ошибку в CV

            if not cv_scores or all(s == np.inf for s in cv_scores) : # Если все фолды пропущены или ошибки
                return {'loss': np.inf, 'status': STATUS_OK}
            avg_rmse = np.mean([s for s in cv_scores if s != np.inf]) # Среднее по успешным фолдам
            if np.isnan(avg_rmse): avg_rmse = np.inf
            return {'loss': avg_rmse, 'status': STATUS_OK}

        trials = Trials()
        best_params_from_fmin = fmin(fn=objective,
                                space=hyperopt_space,
                                algo=tpe.suggest,
                                max_evals=max_evals_hyperopt,
                                trials=trials,
                                rstate=np.random.default_rng(RANDOM_SEED),
                                verbose=0) # Установить verbose=1 или True для отладки fmin

        # Преобразуем лучшие параметры в нужный формат (особенно для hp.choice)
        # и целочисленных параметров из hp.quniform (fmin возвращает float)
        final_best_params = {}
        for key, value in best_params_from_fmin.items():
            # Для hp.quniform, которые должны быть int, fmin вернет float.
            # scope.int используется с hp.quniform для получения int.
            # Если параметр определен как hp.choice с целочисленными опциями, он будет int.
            # Для простоты, если параметр есть в списке "должен быть int", преобразуем
            int_params_list = ['n_estimators', 'max_depth', 'min_samples_leaf', 'min_samples_split', 'num_leaves']
            if key in int_params_list and value is not None:
                final_best_params[key] = int(value)
            else:
                final_best_params[key] = value
        
        # Если в space есть hp.choice, fmin вернет индекс. Нужно получить само значение.
        # Это делается автоматически, если правильно использовать Trials и best_trial.
        # Однако, best_params_from_fmin уже содержит сами значения для простых случаев.
        # Для сложных hp.choice, где опции - другие hp выражения, может потребоваться более сложная логика.
        # Trials.best_trial['misc']['vals'] содержит индексы для hp.choice.
        # Trials.best_trial['result']['params'] может содержать фактические параметры, если вы их возвращаете из objective.
        # Проще всего положиться на то, что `best_params_from_fmin` содержит значения, а не индексы,
        # для большинства случаев, или использовать `hyperopt.space_eval(hyperopt_space, best_params_from_fmin)`
        
        actual_best_params = hyperopt.space_eval(hyperopt_space, best_params_from_fmin)
        # Убедимся, что целочисленные параметры действительно int
        for p_name in ['n_estimators', 'max_depth', 'min_samples_leaf', 'min_samples_split', 'num_leaves']:
            if p_name in actual_best_params and actual_best_params[p_name] is not None:
                actual_best_params[p_name] = int(actual_best_params[p_name])


        best_loss = trials.best_trial['result']['loss'] if trials.best_trial else np.inf
        print(f"    [{component_name_log} - {model_name}] Hyperopt завершен. Лучший CV RMSE: {best_loss:.4f}. Лучшие параметры: {actual_best_params}")

        final_model = model_class(**actual_best_params, random_state=RANDOM_SEED, n_jobs=-1)
        final_model.fit(X_train_comp, y_train_comp_aligned)
        print(f"    [{component_name_log} - {model_name}] Обучен с лучшими параметрами.")

        print(f"    [{component_name_log} - {model_name}] Генерация рекурсивного прогноза...")
        forecast = recursive_predict(final_model, y_train_comp, n_steps,
                                     feature_calculator, feature_order_ml)
        print(f"    [{component_name_log} - {model_name}] Рекурсивный прогноз выполнен.")
        return forecast, actual_best_params

    except Exception as e:
        print(f"    Ошибка [{component_name_log} - {model_name}] (Hyperopt): {e}")
        traceback.print_exc()
        return np.full(n_steps, np.nan), {}


def train_predict_hybrid(y_train_comp, n_steps, arima_order, xgb_hyperopt_space,
                         feature_calculator, feature_order_hybrid, tscv_hybrid, max_evals_hyperopt,
                         component_name_log="Hybrid"):
    """Обучает гибрид ARIMA+XGB_на_ошибках (с Hyperopt для XGB) и делает рекурсивный прогноз."""
    print(f"    [{component_name_log}] Подготовка данных и Hyperopt для XGB на ошибках...")
    try:
        if arima_order is None:
            print(f"      [{component_name_log}] Ошибка: Порядок ARIMA не определен.")
            return np.full(n_steps, np.nan), {}

        print(f"      [{component_name_log}] Обучение базовой ARIMA {arima_order}...")
        arima_model_fit = StatsmodelsARIMA(y_train_comp, order=arima_order,
                                       enforce_stationarity=False, enforce_invertibility=False).fit()
        arima_fitted = arima_model_fit.fittedvalues
        common_idx = y_train_comp.index.intersection(arima_fitted.index)
        if common_idx.empty :
            raise ValueError(f"[{component_name_log}] Нет общих индексов между y_train_comp и arima_fitted.")
        y_train_aligned = y_train_comp.loc[common_idx]
        arima_fitted_aligned = arima_fitted.loc[common_idx]
        arima_errors = y_train_aligned - arima_fitted_aligned
        print(f"      [{component_name_log}] Базовая ARIMA обучена, ошибки рассчитаны (длина {len(arima_errors)}).")

        # Используем y_train_comp для генерации признаков, затем выравниваем с arima_errors
        data_ml_comp_hybrid = pd.DataFrame({'t': y_train_comp})
        for lag in LAGS_TO_CREATE: data_ml_comp_hybrid[f't-{lag}'] = data_ml_comp_hybrid['t'].shift(lag)
        target_series_hybrid = data_ml_comp_hybrid['t']
        for window in ROLLING_WINDOWS:
            data_ml_comp_hybrid[f't_mean_lag{window}'] = target_series_hybrid.rolling(window=window, min_periods=1).mean().shift(1)
            data_ml_comp_hybrid[f't_std_lag{window}'] = target_series_hybrid.rolling(window=window, min_periods=1).std().shift(1)
        if isinstance(data_ml_comp_hybrid.index, pd.DatetimeIndex):
            month_num_hybrid = data_ml_comp_hybrid.index.month
            data_ml_comp_hybrid['month_sin'] = np.sin(2 * np.pi * month_num_hybrid / 12)
            data_ml_comp_hybrid['month_cos'] = np.cos(2 * np.pi * month_num_hybrid / 12)
        else:
            data_ml_comp_hybrid['month_sin'] = np.nan
            data_ml_comp_hybrid['month_cos'] = np.nan
        
        X_train_xgb_hybrid_full = data_ml_comp_hybrid[feature_order_hybrid]
        common_indices_xgb_err = X_train_xgb_hybrid_full.index.intersection(arima_errors.index)
        X_train_xgb_aligned = X_train_xgb_hybrid_full.loc[common_indices_xgb_err].dropna()
        arima_errors_aligned = arima_errors.loc[X_train_xgb_aligned.index]

        min_samples_for_cv_hybrid = 0
        for train_idx_h, _ in tscv_hybrid.split(X_train_xgb_aligned):
            if len(train_idx_h) > min_samples_for_cv_hybrid:
                 min_samples_for_cv_hybrid = len(train_idx_h)

        if X_train_xgb_aligned.empty or len(X_train_xgb_aligned) < tscv_hybrid.get_n_splits() + min_samples_for_cv_hybrid:
             print(f"      [{component_name_log}] Ошибка: Недостаточно данных ({len(X_train_xgb_aligned)}) для Hyperopt XGB (нужно {tscv_hybrid.get_n_splits() + min_samples_for_cv_hybrid}).")
             return np.full(n_steps, np.nan), {}
        print(f"      [{component_name_log}] Признаки для XGB на ошибках готовы ({X_train_xgb_aligned.shape}). Цель (ошибки) готова ({arima_errors_aligned.shape})")

        # --- Hyperopt Objective Function для XGBoost на ошибках ---
        def objective_xgb_error(params):
            current_params = params.copy()
            if 'n_estimators' in current_params: current_params['n_estimators'] = int(current_params['n_estimators'])
            if 'max_depth' in current_params and current_params['max_depth'] is not None: current_params['max_depth'] = int(current_params['max_depth'])
            if 'num_leaves' in current_params: current_params['num_leaves'] = int(current_params['num_leaves'])

            model_xgb_err = xgb.XGBRegressor(**current_params, objective='reg:squarederror', random_state=RANDOM_SEED, n_jobs=-1)
            cv_scores_err = []
            for train_idx, val_idx in tscv_hybrid.split(X_train_xgb_aligned):
                X_cv_train_err, X_cv_val_err = X_train_xgb_aligned.iloc[train_idx], X_train_xgb_aligned.iloc[val_idx]
                y_cv_train_err, y_cv_val_err = arima_errors_aligned.iloc[train_idx], arima_errors_aligned.iloc[val_idx]
                if X_cv_train_err.empty or X_cv_val_err.empty or len(X_cv_train_err) < 2 or len(X_cv_val_err) < 1:
                    continue
                try:
                    model_xgb_err.fit(X_cv_train_err, y_cv_train_err)
                    preds_err = model_xgb_err.predict(X_cv_val_err)
                    rmse_err = np.sqrt(mean_squared_error(y_cv_val_err, preds_err))
                    cv_scores_err.append(rmse_err)
                except Exception:
                    cv_scores_err.append(np.inf)
            if not cv_scores_err or all(s == np.inf for s in cv_scores_err):
                return {'loss': np.inf, 'status': STATUS_OK}
            avg_rmse_err = np.mean([s for s in cv_scores_err if s != np.inf])
            if np.isnan(avg_rmse_err): avg_rmse_err = np.inf
            return {'loss': avg_rmse_err, 'status': STATUS_OK}

        trials_xgb_err = Trials()
        best_params_xgb_err_fmin = fmin(fn=objective_xgb_error,
                                   space=xgb_hyperopt_space, # Используем то же пространство, что и для обычного XGB
                                   algo=tpe.suggest,
                                   max_evals=max_evals_hyperopt,
                                   trials=trials_xgb_err,
                                   rstate=np.random.default_rng(RANDOM_SEED),
                                   verbose=0)

        actual_best_params_xgb_err = hyperopt.space_eval(xgb_hyperopt_space, best_params_xgb_err_fmin)
        for p_name in ['n_estimators', 'max_depth', 'num_leaves']:
             if p_name in actual_best_params_xgb_err and actual_best_params_xgb_err[p_name] is not None:
                actual_best_params_xgb_err[p_name] = int(actual_best_params_xgb_err[p_name])

        best_loss_xgb_err = trials_xgb_err.best_trial['result']['loss'] if trials_xgb_err.best_trial else np.inf
        print(f"      [{component_name_log}] Hyperopt для XGB на ошибках завершен. Лучший CV RMSE: {best_loss_xgb_err:.4f}. Параметры: {actual_best_params_xgb_err}")

        final_xgb_error_model = xgb.XGBRegressor(**actual_best_params_xgb_err, objective='reg:squarederror', random_state=RANDOM_SEED, n_jobs=-1)
        final_xgb_error_model.fit(X_train_xgb_aligned, arima_errors_aligned)
        print(f"      [{component_name_log}] XGB на ошибках обучен с лучшими параметрами.")

        print(f"      [{component_name_log}] Генерация рекурсивного прогноза гибрида...")
        history_arima = list(y_train_comp)
        arima_forecast_rec = []
        for _ in range(n_steps): # Рекурсивный прогноз ARIMA
            model_rec_fit = StatsmodelsARIMA(history_arima, order=arima_order,
                                        enforce_stationarity=False, enforce_invertibility=False).fit()
            next_pred_a = model_rec_fit.forecast(steps=1)[0]
            arima_forecast_rec.append(next_pred_a)
            history_arima.append(next_pred_a)
        arima_forecast_rec = np.array(arima_forecast_rec)
        print(f"        [{component_name_log}] Прогноз ARIMA для гибрида готов.")

        # Рекурсивный прогноз ошибок XGBoost (использует историю y_train_comp для расчета признаков!)
        xgb_error_forecast_rec = recursive_predict(final_xgb_error_model, y_train_comp, n_steps,
                                                  feature_calculator, feature_order_hybrid)
        print(f"        [{component_name_log}] Прогноз ошибок XGB для гибрида готов.")

        final_forecast = arima_forecast_rec + xgb_error_forecast_rec
        print(f"    [{component_name_log}] Рекурсивный прогноз Гибрида выполнен.")
        return final_forecast, {'arima_order': arima_order, 'xgb_error_params': actual_best_params_xgb_err}

    except Exception as e:
        print(f"    Ошибка [{component_name_log}] (Hyperopt): {e}")
        traceback.print_exc()
        return np.full(n_steps, np.nan), {}


# --- 5. Функция для оценки моделей и выбора лучшей (ОБНОВЛЕННАЯ для Hyperopt) ---
def evaluate_component_models_cv(y_train_comp, y_test_comp, component_name, horizons, weights_rmse,
                                 rf_hyperopt_space, xgb_hyperopt_space,
                                 tscv_comp, max_evals_hyperopt):
    model_forecasts = {}
    model_rmses = pd.DataFrame(columns=['model', 'horizon', 'rmse'])
    component_all_model_best_params = {} # Сохраняем лучшие параметры для КАЖДОЙ модели внутри этой компоненты

    # ARIMA
    print(f"  [{component_name}] Модель: ARIMA")
    forecast_arima, order_arima, arima_params = train_predict_arima(y_train_comp, len(y_test_comp))
    model_forecasts['ARIMA'] = forecast_arima
    component_all_model_best_params['ARIMA'] = arima_params
    if order_arima is not None and not np.all(np.isnan(forecast_arima)): # Проверка на все NaN
        for h in horizons:
            if len(y_test_comp[:h]) == len(forecast_arima[:h]):
                 rmse = np.sqrt(mean_squared_error(y_test_comp[:h], forecast_arima[:h]))
                 model_rmses = pd.concat([model_rmses, pd.DataFrame([{'model':'ARIMA', 'horizon':h, 'rmse':rmse}])], ignore_index=True)

    # XGBoost
    print(f"\n  [{component_name}] Модель: XGBoost")
    forecast_xgb, xgb_best_p = train_predict_ml(xgb.XGBRegressor, xgb_hyperopt_space, y_train_comp, len(y_test_comp),
                                        calculate_features_for_step, COMP_FEATURE_ORDER, tscv_comp, max_evals_hyperopt,
                                        component_name_log=component_name) # Передаем имя компоненты для логов
    model_forecasts['XGBoost'] = forecast_xgb
    component_all_model_best_params['XGBoost'] = xgb_best_p
    if not np.all(np.isnan(forecast_xgb)):
        for h in horizons:
            if len(y_test_comp[:h]) == len(forecast_xgb[:h]):
                rmse = np.sqrt(mean_squared_error(y_test_comp[:h], forecast_xgb[:h]))
                model_rmses = pd.concat([model_rmses, pd.DataFrame([{'model':'XGBoost', 'horizon':h, 'rmse':rmse}])], ignore_index=True)

    # RandomForest
    print(f"\n  [{component_name}] Модель: RandomForest")
    forecast_rf, rf_best_p = train_predict_ml(RandomForestRegressor, rf_hyperopt_space, y_train_comp, len(y_test_comp),
                                       calculate_features_for_step, COMP_FEATURE_ORDER, tscv_comp, max_evals_hyperopt,
                                       component_name_log=component_name)
    model_forecasts['RandomForest'] = forecast_rf
    component_all_model_best_params['RandomForest'] = rf_best_p
    if not np.all(np.isnan(forecast_rf)):
        for h in horizons:
            if len(y_test_comp[:h]) == len(forecast_rf[:h]):
                rmse = np.sqrt(mean_squared_error(y_test_comp[:h], forecast_rf[:h]))
                model_rmses = pd.concat([model_rmses, pd.DataFrame([{'model':'RandomForest', 'horizon':h, 'rmse':rmse}])], ignore_index=True)

    # Гибрид ARIMA+XGB_err_Rec
    print(f"\n  [{component_name}] Модель: ARIMA+XGB_err_Rec")
    if order_arima is not None:
        forecast_hybrid, hybrid_best_p = train_predict_hybrid(y_train_comp, len(y_test_comp), order_arima, xgb_hyperopt_space,
                                                  calculate_features_for_step, COMP_FEATURE_ORDER, tscv_comp, max_evals_hyperopt,
                                                  component_name_log=component_name)
        model_forecasts['ARIMA+XGB_err_Rec'] = forecast_hybrid
        component_all_model_best_params['ARIMA+XGB_err_Rec'] = hybrid_best_p
        if not np.all(np.isnan(forecast_hybrid)):
            for h in horizons:
                if len(y_test_comp[:h]) == len(forecast_hybrid[:h]):
                    rmse = np.sqrt(mean_squared_error(y_test_comp[:h], forecast_hybrid[:h]))
                    model_rmses = pd.concat([model_rmses, pd.DataFrame([{'model':'ARIMA+XGB_err_Rec', 'horizon':h, 'rmse':rmse}])], ignore_index=True)
    else:
         print(f"    [{component_name}] Пропуск Гибрида, т.к. ARIMA не обучилась.")
         model_forecasts['ARIMA+XGB_err_Rec'] = np.full(len(y_test_comp), np.nan)
         component_all_model_best_params['ARIMA+XGB_err_Rec'] = {}

    print(f"\n  [{component_name}] Расчет взвешенного RMSE для выбора лучшей модели...")
    weighted_rmses = {}
    models_evaluated = model_rmses['model'].unique()
    for model_name_eval in models_evaluated:
        if model_name_eval not in model_forecasts or np.all(np.isnan(model_forecasts[model_name_eval])):
            print(f"    [{component_name} - {model_name_eval}] Пропуск из-за отсутствия или NaN прогнозов.")
            weighted_rmses[model_name_eval] = np.inf
            continue
        rmses_s = model_rmses[(model_rmses['model'] == model_name_eval) & (model_rmses['horizon'].isin(weights_rmse.index))].set_index('horizon')['rmse'].dropna()
        common_h = rmses_s.index.intersection(weights_rmse.index)
        if not common_h.empty:
            valid_weights = weights_rmse.loc[common_h].dropna()
            final_common_h = rmses_s.loc[common_h].index.intersection(valid_weights.index) # Пересечение после dropna весов
            if not final_common_h.empty:
                weighted_avg_rmse = (rmses_s.loc[final_common_h] * valid_weights.loc[final_common_h]).sum() / valid_weights.loc[final_common_h].sum()
                if np.isnan(weighted_avg_rmse): weighted_avg_rmse = np.inf
                weighted_rmses[model_name_eval] = weighted_avg_rmse
                print(f"    [{component_name} - {model_name_eval}]: Weighted RMSE = {weighted_avg_rmse:.4f} (на {len(final_common_h)} горизонтах)")
            else:
                weighted_rmses[model_name_eval] = np.inf
        else:
            print(f"    [{component_name} - {model_name_eval}]: Недостаточно RMSE или весов для расчета.")
            weighted_rmses[model_name_eval] = np.inf

    best_model_name = "ARIMA" # Запасной вариант
    if not weighted_rmses or all(np.isinf(v) for v in weighted_rmses.values()):
        print(f"  ПРЕДУПРЕЖДЕНИЕ [{component_name}]: Не удалось выбрать лучшую модель. Выбрана ARIMA по умолчанию.")
    else:
        valid_weighted_rmses = {k: v for k, v in weighted_rmses.items() if not (model_forecasts.get(k) is None or np.all(np.isnan(model_forecasts.get(k))))}
        if not valid_weighted_rmses:
             print(f"  ПРЕДУПРЕЖДЕНИЕ [{component_name}]: Нет моделей с валидными прогнозами. Выбрана ARIMA по умолчанию.")
        else:
            best_model_name = min(valid_weighted_rmses, key=valid_weighted_rmses.get)
    print(f"\n  Лучшая модель для компоненты '{component_name}': {best_model_name}")
    return model_forecasts, best_model_name, model_rmses, component_all_model_best_params

print("Вспомогательные функции (обновленные для Hyperopt) определены.")
# --- КОНЕЦ ОБНОВЛЕННОЙ ЯЧЕЙКИ 7 ---


# --- Ячейка 10: Создание Признаков для Мета-моделей на ARIMA OOF ---
# Логика этой ячейки не меняется, т.к. она использует create_meta_features,
# которая не зависит от GridSearchCV или Hyperopt.
# Убедитесь, что все необходимые переменные (df_oof_comp_arima_clean,
# train_values_aligned_meta_arima, df_forecasts_arima_comp, COMPONENTS_TO_MODEL,
# LAGS_TO_CREATE, ROLLING_WINDOWS, META_FEATURE_ORDER) определены корректно.
print("\nЯчейка 10: Создание Признаков для Мета-моделей на ARIMA OOF")
# ... (ваш код Ячейки 10 без изменений) ...
# --- ПРЕДПОЛАГАЕМ НАЛИЧИЕ ПЕРЕМЕННЫХ ---
# df_oof_comp_arima_clean: DataFrame с OOF ARIMA прогнозами (чистый)
# train_values_aligned_meta_arima: Series с таргетом (агрегат), выровненным с OOF ARIMA
# df_forecasts_arima_comp: DataFrame с ARIMA прогнозами компонент на тесте
# components_to_model: list
# create_meta_features: function (должна быть определена в Ячейке 7)
# META_FEATURE_ORDER: list - порядок признаков для мета-моделей (определен в Ячейке 7)
# LAGS_TO_CREATE, ROLLING_WINDOWS: list (определены в Ячейке 2 или 7)
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

# --- 1. Создание признаков для ОБУЧАЮЩЕЙ выборки ---
# print("Создание признаков для ОБУЧАЮЩЕЙ выборки (на ARIMA OOF)...")
# try:
#     X_train_meta_arima_eng = create_meta_features(
#         df_input=df_oof_comp_arima_clean,
#         component_names=COMPONENTS_TO_MODEL,
#         lags=LAGS_TO_CREATE,
#         windows=ROLLING_WINDOWS,
#         feature_order_meta=META_FEATURE_ORDER
#     )
#     X_train_meta_arima_eng_clean = X_train_meta_arima_eng.dropna()
#     y_train_meta_arima_aligned = train_values_aligned_meta_arima.loc[X_train_meta_arima_eng_clean.index]
#     print(f"Размер обучающей выборки с признаками (ARIMA OOF): {X_train_meta_arima_eng_clean.shape}")
#     print(f"Размер выровненного таргета: {y_train_meta_arima_aligned.shape}")
# except Exception as e:
#     print(f"Ошибка при создании обучающих признаков для мета-модели: {e}")
#     X_train_meta_arima_eng_clean = None
#     y_train_meta_arima_aligned = None

# # --- 2. Создание признаков для ТЕСТОВОЙ выборки ---
# if X_train_meta_arima_eng_clean is not None:
#     print("\nСоздание признаков для ТЕСТОВОЙ выборки (на ARIMA прогнозах теста)...")
#     try:
#         max_lookback_arima = max(max(LAGS_TO_CREATE, default=0), max(ROLLING_WINDOWS, default=0))
#         if len(df_oof_comp_arima_clean) >= max_lookback_arima:
#             history_for_test_arima = df_oof_comp_arima_clean.iloc[-max_lookback_arima:]
#         else:
#             print(f"Предупреждение: Недостаточно OOF истории ({len(df_oof_comp_arima_clean)}) для lookback ({max_lookback_arima}). Используется вся доступная OOF история.")
#             history_for_test_arima = df_oof_comp_arima_clean.copy()
#         df_for_test_features_arima = pd.concat([history_for_test_arima, df_forecasts_arima_comp])
#         X_test_meta_arima_eng_full = create_meta_features(
#             df_input=df_for_test_features_arima,
#             component_names=COMPONENTS_TO_MODEL,
#             lags=LAGS_TO_CREATE,
#             windows=ROLLING_WINDOWS,
#             feature_order_meta=META_FEATURE_ORDER
#         )
#         X_test_meta_arima_eng = X_test_meta_arima_eng_full.loc[df_forecasts_arima_comp.index]
#         if X_test_meta_arima_eng.isnull().any().any():
#             print("ВНИМАНИЕ: Обнаружены NaN в тестовых признаках (ARIMA OOF). Заполняем медианой трейна...")
#             for col in X_test_meta_arima_eng.columns:
#                 if X_test_meta_arima_eng[col].isnull().any():
#                     median_val = X_train_meta_arima_eng_clean[col].median()
#                     X_test_meta_arima_eng[col] = X_test_meta_arima_eng[col].fillna(median_val)
#             print(f"Кол-во NaN после заполнения: {X_test_meta_arima_eng.isnull().sum().sum()}")
#         else:
#             print("NaN в тестовых признаках (ARIMA OOF) не обнаружены.")
#         print("\n--- Признаки для мета-моделей на ARIMA OOF готовы ---")
#         print(f"Размер X_train_meta_arima_eng_clean: {X_train_meta_arima_eng_clean.shape}")
#         print(f"Размер y_train_meta_arima_aligned: {y_train_meta_arima_aligned.shape}")
#         print(f"Размер X_test_meta_arima_eng: {X_test_meta_arima_eng.shape}")
#     except Exception as e:
#         print(f"Ошибка при создании тестовых признаков для мета-модели: {e}")
#         X_test_meta_arima_eng = None
# else:
#      print("Пропуск создания тестовых признаков из-за ошибки на этапе создания обучающих.")
#      X_test_meta_arima_eng = None
# print("\n--- Ячейка 10 завершена ---")
# print("Колонки X_train_meta_arima_eng:", X_train_meta_arima_eng.columns.tolist() if X_train_meta_arima_eng is not None else "N/A")
# print("Количество колонок:", len(X_train_meta_arima_eng.columns) if X_train_meta_arima_eng is not None else "N/A")
# print("Размер до dropna:", X_train_meta_arima_eng.shape if X_train_meta_arima_eng is not None else "N/A")

# --- КОНЕЦ ЯЧЕЙКИ 10 ---


# --- Ячейка 14: Запуск Оценки Моделей для Каждой Компоненты (с Hyperopt для ML) ---
print("\n--- Раздел 4: Bottom-up Подход - Выбор Лучшей Модели для Компонент ---")
print("\nЯчейка 14: Запуск Оценки Моделей для Каждой Компоненты (с Hyperopt для ML)")

# --- ПРЕДПОЛАГАЕМ НАЛИЧИЕ ГЛОБАЛЬНЫХ ПЕРЕМЕННЫХ ---
# train_comp_data, test_comp_data: DataFrames
# COMPONENTS_TO_MODEL: list
# HORIZONS_TO_EVALUATE: list
# WEIGHTS_NORMALIZED_RMSE: pd.Series
# N_CV_SPLITS: int
# MAX_EVALS_HYPEROPT: int
# COMP_FEATURE_ORDER: list
# LAGS_TO_CREATE, ROLLING_WINDOWS
# RANDOM_SEED
# TEST_MONTHS
# tscv_component_cv: TimeSeriesSplit объект
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ --

# Определяем "обоснованные" пространства поиска для Hyperopt
# hp.quniform(label, low, high, q) -> round(uniform(low, high) / q) * q
# Для строго целочисленных значений используем scope.int()
rf_hyperopt_space_comp = {
    'n_estimators': scope.int(hp.quniform('n_estimators_rf', 50, 250, 25)),
    'max_depth': hp.choice('max_depth_rf', [None, scope.int(hp.quniform('max_depth_val_rf', 3, 15, 1))]),
    'max_features': hp.choice('max_features_rf', ['sqrt', 'log2', hp.uniform('max_features_float_rf', 0.4, 0.9)]),
    'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf_rf', 1, 5, 1)),
    'min_samples_split': scope.int(hp.quniform('min_samples_split_rf', 2, 10, 1))
}

xgb_hyperopt_space_comp = {
    'n_estimators': scope.int(hp.quniform('n_estimators_xgb', 50, 350, 25)),
    'max_depth': scope.int(hp.quniform('max_depth_xgb', 2, 8, 1)),
    'learning_rate': hp.loguniform('learning_rate_xgb', np.log(0.005), np.log(0.2)),
    'subsample': hp.uniform('subsample_xgb', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree_xgb', 0.5, 1.0),
    'gamma': hp.uniform('gamma_xgb', 0, 0.7),
    'reg_alpha': hp.uniform('reg_alpha_xgb', 0, 0.5), # L1
    'reg_lambda': hp.uniform('reg_lambda_xgb', 0, 0.5), # L2
    # 'num_leaves': scope.int(hp.quniform('num_leaves_xgb', 10, 100, 5)) # Если используется LightGBM или XGB с tree_method='hist'
}

# Инициализируем структуры для хранения результатов
component_best_models = {} # {компонента: имя_лучшей_модели}
component_best_forecasts_dict = {} # {компонента: pd.Series прогноза}
all_component_model_rmses_list = [] # список DataFrame'ов RMSE для всех моделей всех компонент
all_components_best_params = {} # {компонента: {модель: параметры}}

print(f"Запуск оценки {len(COMPONENTS_TO_MODEL)} компонент...")
print(f"Используется TimeSeriesSplit с {N_CV_SPLITS} сплитами для Hyperopt ML моделей (max_evals={MAX_EVALS_HYPEROPT}).")

for component_name in COMPONENTS_TO_MODEL:
    print(f"\n===== Обработка компоненты: {component_name} =====")
    y_train_c = train_comp_data[component_name].copy()
    if not isinstance(y_train_c.index, pd.DatetimeIndex):
        y_train_c.index = pd.to_datetime(y_train_c.index)
    y_train_c = y_train_c.rename('t').dropna()

    y_test_c = test_comp_data[component_name].copy()
    if not isinstance(y_test_c.index, pd.DatetimeIndex):
        y_test_c.index = pd.to_datetime(y_test_c.index)
    y_test_c = y_test_c.rename('t')

    if len(y_train_c) < 20: # Минимальный размер обучающей выборки
        print(f"  ПРЕДУПРЕЖДЕНИЕ: Слишком мало данных ({len(y_train_c)}) для компоненты {component_name}. Пропуск.")
        component_best_models[component_name] = 'Skipped'
        component_best_forecasts_dict[component_name] = pd.Series(np.full(len(y_test_c), np.nan), index=y_test_c.index)
        all_components_best_params[component_name] = {} # Нет параметров для пропущенной компоненты
        continue

    model_forecasts_comp, best_model_name_comp, component_rmses_df, comp_model_p = evaluate_component_models_cv(
        y_train_c,
        y_test_c,
        component_name,
        HORIZONS_TO_EVALUATE,
        WEIGHTS_NORMALIZED_RMSE,
        rf_hyperopt_space_comp,
        xgb_hyperopt_space_comp,
        tscv_component_cv, # Передаем объект TimeSeriesSplit
        MAX_EVALS_HYPEROPT
    )

    component_best_models[component_name] = best_model_name_comp
    all_components_best_params[component_name] = comp_model_p # Сохраняем параметры всех моделей для этой компоненты

    if best_model_name_comp and best_model_name_comp in model_forecasts_comp and model_forecasts_comp[best_model_name_comp] is not None:
        best_fc = model_forecasts_comp[best_model_name_comp]
        # Приводим к pd.Series с правильным индексом и длиной
        final_fc_series = pd.Series(np.nan, index=y_test_c.index)
        common_len = min(len(best_fc), len(y_test_c))
        final_fc_series.iloc[:common_len] = best_fc[:common_len]
        component_best_forecasts_dict[component_name] = final_fc_series

        if len(best_fc) != len(y_test_c):
            print(f"  Предупреждение для {component_name} ({best_model_name_comp}): Длина прогноза ({len(best_fc)}) не совпадает с тестом ({len(y_test_c)}). Скорректировано.")
    else:
        print(f"  ПРЕДУПРЕЖДЕНИЕ для {component_name}: Не удалось получить прогноз лучшей модели ({best_model_name_comp}) или он NaN. Заполняем NaN.")
        component_best_forecasts_dict[component_name] = pd.Series(np.full(len(y_test_c), np.nan), index=y_test_c.index)

    if not component_rmses_df.empty:
        component_rmses_df['component'] = component_name
        all_component_model_rmses_list.append(component_rmses_df)

print("\n--- Выбор лучших моделей для компонент завершен ---")

df_best_component_forecasts = pd.DataFrame(component_best_forecasts_dict)
if not df_best_component_forecasts.empty and not isinstance(df_best_component_forecasts.index, pd.DatetimeIndex):
    # Попытка восстановить DatetimeIndex, если он был утерян (например, если dict был пуст и создался с RangeIndex)
    if not test_comp_data.index.empty:
         df_best_component_forecasts.index = test_comp_data.index[:len(df_best_component_forecasts)]


if all_component_model_rmses_list:
    df_all_component_rmses = pd.concat(all_component_model_rmses_list, ignore_index=True)
else:
    df_all_component_rmses = pd.DataFrame(columns=['model', 'horizon', 'rmse', 'component'])

print("\n--- Лучшие параметры, найденные Hyperopt для каждой компоненты и модели: ---")
for comp_name_p, models_p_dict in all_components_best_params.items():
    print(f"  Компонента: {comp_name_p}")
    if models_p_dict:
        for model_n, params_m in models_p_dict.items():
            print(f"    Модель: {model_n}, Параметры: {params_m}")
    else:
        print("    Параметры не найдены (компонент мог быть пропущен или для него не нашлись параметры).")

print("\n--- Ячейка 14 завершена ---")

In [ ]:
print("\nЯчейка 15: Анализ Выбора Моделей Компонент")

# --- ПРЕДПОЛАГАЕМ НАЛИЧИЕ ПЕРЕМЕННЫХ ---
# component_best_models: dict - Словарь с именами лучших моделей для компонент
# df_all_component_rmses: pd.DataFrame - Таблица с RMSE всех моделей для всех компонент
# df_best_component_forecasts: pd.DataFrame - Прогнозы лучших моделей на тесте
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

# 1. Вывод списка лучших моделей
print("--- Лучшие модели, выбранные для каждой компоненты ---")
if 'component_best_models' in locals() and component_best_models:
    for component, model_name in component_best_models.items():
        print(f"  - {component}: {model_name}")
else:
    print("Словарь лучших моделей пуст или не определен.")

# 2. Вывод сводной таблицы RMSE всех моделей по компонентам
print("\n--- Сводная таблица RMSE моделей по компонентам и горизонтам ---")
if 'df_all_component_rmses' in locals() and not df_all_component_rmses.empty:
    try:
        # Создаем сводную таблицу: строки - компонента и горизонт, колонки - модели
        pivot_comp_rmse = df_all_component_rmses.pivot_table(
            index=['component', 'horizon'],
            columns='model',
            values='rmse'
        )
        # Определяем порядок колонок (моделей) для вывода
        model_order = ['ARIMA', 'RandomForest', 'XGBoost', 'ARIMA+XGB_err_Rec'] # Примерный порядок
        # Убираем модели, которых нет в таблице
        model_order_present = [m for m in model_order if m in pivot_comp_rmse.columns]
        # Добавляем остальные модели, если они есть
        remaining_models = [m for m in pivot_comp_rmse.columns if m not in model_order_present]
        final_model_order = model_order_present + remaining_models

        print(pivot_comp_rmse[final_model_order].round(4)) # Выводим с заданным порядком
    except KeyError as e:
        print(f"Ошибка при создании сводной таблицы (возможно, отсутствует колонка): {e}")
        print("Вывод таблицы как есть:")
        try:
            print(df_all_component_rmses.pivot_table(index=['component', 'horizon'], columns='model', values='rmse').round(4))
        except Exception as e_inner:
            print(f"Не удалось вывести таблицу: {e_inner}")
            print("Данные RMSE:")
            print(df_all_component_rmses) # Печатаем сырые данные для отладки
    except Exception as e:
        print(f"Не удалось создать или вывести сводную таблицу RMSE: {e}")
        print("Данные RMSE:")
        print(df_all_component_rmses)
else:
    print("Нет данных по RMSE моделей компонент для отображения.")

# 3. Вывод начала таблицы с лучшими прогнозами
print("\n--- Лучшие прогнозы для агрегации (первые 5 строк) ---")
if 'df_best_component_forecasts' in locals() and not df_best_component_forecasts.empty:
    print(df_best_component_forecasts.head())
    print("\nПроверка на NaN в лучших прогнозах:")
    print(df_best_component_forecasts.isnull().sum())
else:
    print("DataFrame с лучшими прогнозами пуст или не определен.")

print("\n--- Ячейка 15 завершена ---")

In [ ]:
print("\n--- Раздел 5: Агрегация Лучших Прогнозов Компонент ---")
print("\nЯчейка 16: Агрегация Лучших Прогнозов Простыми Весами")

# --- ПРЕДПОЛАГАЕМ НАЛИЧИЕ ПЕРЕМЕННЫХ ---
# df_best_component_forecasts: DataFrame [ИндексТеста x Компоненты] - прогнозы ЛУЧШИХ моделей (из Ячейки 14)
# weights_norm: DataFrame [Год x Компонент] - нормализованные веса (из Ячейки 4)
# train_values: Series - Обучающая выборка агрегата (из Ячейки 5)
# test_values: Series - Тестовая выборка агрегата (из Ячейки 5)
# COMPONENTS_TO_MODEL: list
# HORIZONS_TO_EVALUATE: list
# aggregation_rmse: dict (инициализирован в Ячейке 2)
# all_horizon_rmse: pd.DataFrame (инициализирован в Ячейке 2)
# final_forecasts: dict (инициализирован в Ячейке 2)
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

# Проверка на NaN в лучших прогнозах (важно!)
if 'df_best_component_forecasts' not in locals() or df_best_component_forecasts is None:
     print("ОШИБКА: DataFrame 'df_best_component_forecasts' не найден. Запустите Ячейку 14.")
     # Останавливаем выполнение или обрабатываем ошибку
     raise NameError("df_best_component_forecasts не определен")
elif df_best_component_forecasts.isnull().any().any():
    print("ВНИМАНИЕ: Обнаружены NaN в 'df_best_component_forecasts'. Результаты агрегации могут быть некорректными.")
    print(df_best_component_forecasts.isnull().sum())
    # Можно добавить стратегию обработки NaN, если необходимо
    # df_best_component_forecasts = df_best_component_forecasts.fillna(method='ffill').fillna(method='bfill') # Пример

# --- Метод 1: Веса последнего года обучения ---
model_name_best_lastw = "BottomUp-BestComp-LastW" # Используем это имя
print(f"\n--- Расчет: {model_name_best_lastw} ---")
try:
    last_train_date_agg = train_values.index.max()
    last_train_year_agg = last_train_date_agg.year

    if last_train_year_agg in weights_norm.index:
        last_train_weights_agg = weights_norm.loc[last_train_year_agg]
        print(f"Используются веса за {last_train_year_agg} год:\n", last_train_weights_agg[COMPONENTS_TO_MODEL])

        # Агрегация (убедимся, что порядок колонок совпадает)
        forecast_best_lastw = (df_best_component_forecasts[COMPONENTS_TO_MODEL] * last_train_weights_agg[COMPONENTS_TO_MODEL]).sum(axis=1)
        final_forecasts[model_name_best_lastw] = forecast_best_lastw.values

        # Оценка
        eval_len = min(len(forecast_best_lastw), len(test_values))
        overall_rmse = np.sqrt(mean_squared_error(test_values[:eval_len], forecast_best_lastw[:eval_len]))
        print(f"Общий RMSE: {overall_rmse:.4f}")
        aggregation_rmse[model_name_best_lastw] = overall_rmse

        print("RMSE по горизонтам:")
        horizon_results_list = []
        for h in HORIZONS_TO_EVALUATE:
            if h <= eval_len:
                rmse_h = np.sqrt(mean_squared_error(test_values[:h], forecast_best_lastw[:h]))
                # print(f"  1-{h} мес.: {rmse_h:.4f}") # Убрал вывод для краткости
                horizon_results_list.append({'model': model_name_best_lastw, 'horizon': h, 'rmse': rmse_h})
        if horizon_results_list:
             df_horizon = pd.DataFrame(horizon_results_list)
             all_horizon_rmse = all_horizon_rmse[all_horizon_rmse['model'] != model_name_best_lastw]
             all_horizon_rmse = pd.concat([all_horizon_rmse, df_horizon], ignore_index=True)
        print("RMSE по горизонтам рассчитаны.")

    else:
        print(f"ОШИБКА: Веса за {last_train_year_agg} год не найдены!")
        aggregation_rmse[model_name_best_lastw] = np.nan

except Exception as e:
    print(f"Ошибка при расчете {model_name_best_lastw}: {e}")
    aggregation_rmse[model_name_best_lastw] = np.nan


# --- Метод 2: Средние веса ---
model_name_best_avgw = "BottomUp-BestComp-AvgW" # Используем это имя
print(f"\n--- Расчет: {model_name_best_avgw} ---")
try:
    avg_weights_agg = weights_norm.mean()
    print("Используемые средние веса:\n", avg_weights_agg[COMPONENTS_TO_MODEL])

    # Агрегация
    forecast_best_avgw = (df_best_component_forecasts[COMPONENTS_TO_MODEL] * avg_weights_agg[COMPONENTS_TO_MODEL]).sum(axis=1)
    final_forecasts[model_name_best_avgw] = forecast_best_avgw.values

    # Оценка
    eval_len = min(len(forecast_best_avgw), len(test_values))
    overall_rmse = np.sqrt(mean_squared_error(test_values[:eval_len], forecast_best_avgw[:eval_len]))
    print(f"Общий RMSE: {overall_rmse:.4f}")
    aggregation_rmse[model_name_best_avgw] = overall_rmse

    print("RMSE по горизонтам:")
    horizon_results_list = []
    for h in HORIZONS_TO_EVALUATE:
         if h <= eval_len:
            rmse_h = np.sqrt(mean_squared_error(test_values[:h], forecast_best_avgw[:h]))
            # print(f"  1-{h} мес.: {rmse_h:.4f}") # Убрал вывод
            horizon_results_list.append({'model': model_name_best_avgw, 'horizon': h, 'rmse': rmse_h})
    if horizon_results_list:
        df_horizon = pd.DataFrame(horizon_results_list)
        all_horizon_rmse = all_horizon_rmse[all_horizon_rmse['model'] != model_name_best_avgw]
        all_horizon_rmse = pd.concat([all_horizon_rmse, df_horizon], ignore_index=True)
    print("RMSE по горизонтам рассчитаны.")

except Exception as e:
    print(f"Ошибка при расчете {model_name_best_avgw}: {e}")
    aggregation_rmse[model_name_best_avgw] = np.nan


# --- Вывод обновленных таблиц ---
print("\n--- Обновленное сравнение RMSE (Общий RMSE) ---")
if 'aggregation_rmse' in locals() and aggregation_rmse:
    # Добавляем проверку, что ключи существуют перед сортировкой
    valid_keys = [k for k in aggregation_rmse if pd.notna(aggregation_rmse[k])]
    if valid_keys:
        sorted_rmse = {k: aggregation_rmse[k] for k in sorted(valid_keys, key=aggregation_rmse.get)}
        for model, rmse in sorted_rmse.items(): print(f"  {model}: {rmse:.4f}")
    else: print("Нет валидных RMSE для сортировки.")
else: print("Словарь aggregation_rmse пуст.")

print("\nОбновленная таблица RMSE по горизонтам:")
if 'all_horizon_rmse' in locals() and not all_horizon_rmse.empty:
    try:
        pivot_rmse_final = all_horizon_rmse.pivot(index='horizon', columns='model', values='rmse')
        # Получаем порядок из обновленного aggregation_rmse
        if 'sorted_rmse' in locals():
             ordered_columns_keys = list(sorted_rmse.keys())
             ordered_columns_final = [col for col in ordered_columns_keys if col in pivot_rmse_final.columns]
             remaining_cols_final = [col for col in pivot_rmse_final.columns if col not in ordered_columns_final]
             pivot_rmse_final = pivot_rmse_final[ordered_columns_final + remaining_cols_final]
        print(pivot_rmse_final.round(4))
    except Exception as e: print(f"Не удалось создать сводную таблицу: {e}")
else: print("Нет данных.")

print("\n--- Ячейка 16 завершена ---")

In [ ]:
print("\nЯчейка 17: Генерация OOF-прогнозов от Лучших Моделей (с Hyperopt)")

# --- ПРЕДПОЛАГАЕМ НАЛИЧИЕ ПЕРЕМЕННЫХ ---
# train_comp_data: DataFrame
# components_to_model: list
# component_best_models: Словарь {компонента: имя_лучшей_модели} (из Ячейки 14/15)
# N_OOF_SPLITS: int
# N_CV_SPLITS: int (для внутреннего CV в Hyperopt)
# MAX_EVALS_HYPEROPT: int (для Hyperopt)
# Функции: calculate_features_for_step, recursive_predict (из Ячейки 7)
# Функции: train_predict_arima, train_predict_ml, train_predict_hybrid (из Ячейки 7, обновленные для Hyperopt)
# Пространства поиска: rf_hyperopt_space_comp, xgb_hyperopt_space_comp (из Ячейки 14)
# Порядки: arima_orders_dict (сформированный словарь {компонента: порядок_ARIMA})
# COMP_FEATURE_ORDER: list
# Глобальные переменные, используемые функциями (RANDOM_SEED, LAGS_TO_CREATE, ROLLING_WINDOWS)
# train_values: pd.Series (агрегированный таргет для финального выравнивания)
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

# --- Инициализация DataFrame для OOF прогнозов лучших моделей ---
df_oof_best_comp = pd.DataFrame(index=train_comp_data.index,
                                columns=COMPONENTS_TO_MODEL, # Используем COMPONENTS_TO_MODEL
                                dtype=float)

print(f"Генерация OOF-прогнозов ЛУЧШИХ моделей (согласно component_best_models)...")
print(f"Используется TimeSeriesSplit с {N_OOF_SPLITS} фолдами для OOF.")
print(f"Для ML моделей внутри каждого OOF фолда будет запущен Hyperopt (max_evals={MAX_EVALS_HYPEROPT}) с {N_CV_SPLITS} внутренними CV фолдами.")
# print(f"Выбранные лучшие модели: {component_best_models}") # Раскомментируйте, если нужно детально посмотреть

# Инициализируем TimeSeriesSplit для OOF
tscv_oof_best = TimeSeriesSplit(n_splits=N_OOF_SPLITS)
# Инициализируем TimeSeriesSplit для CV внутри ML моделей (Hyperopt)
tscv_inner_cv_oof = TimeSeriesSplit(n_splits=N_CV_SPLITS) # Переименовал во избежание путаницы с tscv_component_cv из Ячейки 14

# --- Цикл по компонентам ---
for i, component_name in enumerate(COMPONENTS_TO_MODEL): # Используем COMPONENTS_TO_MODEL
    print(f"\n({i+1}/{len(COMPONENTS_TO_MODEL)}) OOF для компоненты: {component_name}")

    # Убедимся, что индекс является DatetimeIndex перед работой
    component_series_train_original = train_comp_data[component_name].copy()
    if not isinstance(component_series_train_original.index, pd.DatetimeIndex):
        component_series_train_original.index = pd.to_datetime(component_series_train_original.index)
    component_series_train = component_series_train_original.rename('t').dropna()

    best_model_type = component_best_models.get(component_name)

    if not best_model_type or best_model_type == 'Skipped': # Проверка на пропущенные модели
        print(f"  ПРЕДУПРЕЖДЕНИЕ: Лучшая модель для {component_name} не определена или пропущена ({best_model_type}). Пропуск OOF.")
        df_oof_best_comp[component_name] = np.nan # Заполняем NaN, если модель была пропущена
        continue

    print(f"  Лучшая модель для OOF: {best_model_type}")

    oof_predictions_component = pd.Series(index=component_series_train.index, dtype=float)
    current_arima_order = arima_orders_dict.get(component_name)

    split_count = 0
    for train_idx, val_idx in tscv_oof_best.split(component_series_train):
        split_count += 1
        train_data_split = component_series_train.iloc[train_idx]
        # val_data_split = component_series_train.iloc[val_idx] # Не используется напрямую, только индексы
        val_indices = component_series_train.index[val_idx]
        n_steps_val = len(val_idx)

        if len(train_data_split) < 20: # Минимальный размер обучающей выборки на фолде
             print(f"    Фолд {split_count}/{N_OOF_SPLITS}: Пропуск (мало данных: {len(train_data_split)}) для {best_model_type}")
             # oof_predictions_component.loc[val_indices] = np.nan # Уже инициализировано NaN по умолчанию
             continue
        else:
             print(f"    Фолд {split_count}/{N_OOF_SPLITS}: Обучение {best_model_type} на {len(train_data_split)}, прогноз на {n_steps_val} шагов.")

        forecast_val_fold = np.full(n_steps_val, np.nan) # Инициализация прогноза для текущего фолда

        try:
            if best_model_type == 'ARIMA':
                forecast_val_fold, _, _ = train_predict_arima(train_data_split, n_steps_val) # train_predict_arima возвращает (forecast, order, params)

            elif best_model_type == 'RandomForest':
                forecast_val_fold, _ = train_predict_ml( # train_predict_ml возвращает (forecast, params)
                    model_class=RandomForestRegressor,
                    hyperopt_space=rf_hyperopt_space_comp, # Пространство Hyperopt для RF
                    y_train_comp=train_data_split,
                    n_steps=n_steps_val,
                    feature_calculator=calculate_features_for_step,
                    feature_order_ml=COMP_FEATURE_ORDER,
                    tscv_ml=tscv_inner_cv_oof, # TS Splitter для внутреннего CV Hyperopt
                    max_evals_hyperopt=MAX_EVALS_HYPEROPT,
                    component_name_log=f"{component_name}_OOF_Fold{split_count}"
                )

            elif best_model_type == 'XGBoost':
                 forecast_val_fold, _ = train_predict_ml( # train_predict_ml возвращает (forecast, params)
                    model_class=xgb.XGBRegressor,
                    hyperopt_space=xgb_hyperopt_space_comp, # Пространство Hyperopt для XGB
                    y_train_comp=train_data_split,
                    n_steps=n_steps_val,
                    feature_calculator=calculate_features_for_step,
                    feature_order_ml=COMP_FEATURE_ORDER,
                    tscv_ml=tscv_inner_cv_oof,
                    max_evals_hyperopt=MAX_EVALS_HYPEROPT,
                    component_name_log=f"{component_name}_OOF_Fold{split_count}"
                )

            elif best_model_type == 'ARIMA+XGB_err_Rec':
                if current_arima_order:
                    forecast_val_fold, _ = train_predict_hybrid( # train_predict_hybrid возвращает (forecast, params)
                        y_train_comp=train_data_split,
                        n_steps=n_steps_val,
                        arima_order=current_arima_order,
                        xgb_hyperopt_space=xgb_hyperopt_space_comp, # Пространство Hyperopt для XGB в гибриде
                        feature_calculator=calculate_features_for_step,
                        feature_order_hybrid=COMP_FEATURE_ORDER,
                        tscv_hybrid=tscv_inner_cv_oof,
                        max_evals_hyperopt=MAX_EVALS_HYPEROPT,
                        component_name_log=f"{component_name}_OOF_Fold{split_count}"
                    )
                else:
                    print(f"      Предупреждение: Порядок ARIMA для гибрида ({component_name}) не найден в arima_orders_dict на фолде {split_count}. Прогноз будет NaN.")
                    # forecast_val_fold остается np.full(n_steps_val, np.nan)

            else:
                print(f"    Неизвестный тип модели: {best_model_type} для компоненты {component_name}. Прогноз будет NaN.")
                # forecast_val_fold остается np.full(n_steps_val, np.nan)

            # Сохраняем прогноз фолда
            if forecast_val_fold is not None and not np.all(np.isnan(forecast_val_fold)): # Проверяем, что прогноз не полностью NaN
                 # Обрезаем или дополняем прогноз до нужной длины n_steps_val, если необходимо
                actual_len_forecast = len(forecast_val_fold)
                if actual_len_forecast >= n_steps_val:
                    oof_predictions_component.loc[val_indices] = forecast_val_fold[:n_steps_val]
                else: # Прогноз короче, чем нужно
                    # print(f"      Предупреждение: Прогноз на фолде {split_count} для {best_model_type} ({component_name}) короче ({actual_len_forecast}), чем ожидалось ({n_steps_val}). Дополняем NaN.")
                    padded_val = np.full(n_steps_val, np.nan)
                    padded_val[:actual_len_forecast] = forecast_val_fold
                    oof_predictions_component.loc[val_indices] = padded_val
            # else:
                # print(f"      Предупреждение: Прогноз на фолде {split_count} для {best_model_type} ({component_name}) пуст или полностью NaN.")

        except Exception as e:
            print(f"    ОШИБКА на фолде {split_count} для {component_name} ({best_model_type}): {e}")
            traceback.print_exc()
            # oof_predictions_component.loc[val_indices] остается NaN для этого фолда

    # Сохраняем OOF-прогнозы для всей компоненты
    # Нужно убедиться, что oof_predictions_component имеет тот же индекс, что и колонка в df_oof_best_comp
    df_oof_best_comp.loc[oof_predictions_component.index, component_name] = oof_predictions_component
    print(f"  OOF-прогнозы для {component_name} ({best_model_type}) сгенерированы. Не NaN: {oof_predictions_component.notna().sum()}/{len(oof_predictions_component)}")

print("\n--- Генерация OOF-прогнозов лучших моделей завершена ---")

# --- Постобработка и проверка OOF ---
print("\nКоличество НЕ-NaN OOF-прогнозов (лучшие модели) по компонентам:")
print(df_oof_best_comp.notna().sum(axis=0)) # Сумма по колонкам
print(f"Всего НЕ-NaN OOF-прогнозов: {df_oof_best_comp.notna().sum().sum()}")

# Удаляем строки, где ВСЕ компоненты имеют NaN OOF прогнозы
# Это важно, так как если одна компонента дала NaN, а другие нет, строку не нужно удалять
df_oof_best_comp_clean = df_oof_best_comp.dropna(how='all')


if df_oof_best_comp_clean.empty:
     print("\nОШИБКА: После удаления строк, где все прогнозы NaN, не осталось OOF-прогнозов лучших моделей.")
     # Дальнейший код обучения мета-модели выполнять не стоит
     train_values_aligned_meta = pd.Series(dtype=float) # Пустая серия
else:
    # Выравниваем таргет для МЕТА-модели (используем train_values из Раздела 1)
    # Убедимся, что train_values также имеет DatetimeIndex
    if not isinstance(train_values.index, pd.DatetimeIndex):
        train_values.index = pd.to_datetime(train_values.index)

    common_index_meta = df_oof_best_comp_clean.index.intersection(train_values.index)
    df_oof_best_comp_clean = df_oof_best_comp_clean.loc[common_index_meta]
    train_values_aligned_meta = train_values.loc[common_index_meta]

    print(f"\nРазмер OOF DataFrame лучших моделей после очистки и выравнивания: {df_oof_best_comp_clean.shape}")
    print(f"Размер выровненного таргета для мета-модели: {train_values_aligned_meta.shape}")

    if not df_oof_best_comp_clean.empty:
        print("\nПервые 5 строк OOF-прогнозов (лучшие модели, очищенные):")
        print(df_oof_best_comp_clean.head())
    else:
        print("\nOOF DataFrame лучших моделей пуст после очистки и выравнивания.")


# Теперь df_oof_best_comp_clean и train_values_aligned_meta готовы для Ячейки 18 (Признаки) и 19 (Meta-RF).
print("\n--- Ячейка 17 завершена ---")

In [ ]:
print("\nЯчейка 18: Создание Признаков для Мета-модели на Лучших OOF")

# --- ПРЕДПОЛАГАЕМ НАЛИЧИЕ ПЕРЕМЕННЫХ ---
# df_oof_best_comp_clean: DataFrame с OOF лучших моделей (чистый)
# train_values_aligned_meta: Series с таргетом (агрегат), выровненным с ЭТИМИ OOF
# df_best_component_forecasts: DataFrame с прогнозами лучших моделей на тесте
# components_to_model: list
# create_meta_features: function (из Ячейки 7)
# META_FEATURE_ORDER: list - порядок признаков для мета-моделей (из Ячейки 7, 35 признаков)
# LAGS_TO_CREATE, ROLLING_WINDOWS: list
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

# --- 1. Создание признаков для ОБУЧАЮЩЕЙ выборки ---
print("Создание признаков для ОБУЧАЮЩЕЙ выборки (на OOF лучших моделей)...")
try:
    # Применяем функцию к OOF прогнозам лучших моделей
    X_train_meta_best_eng = create_meta_features(
        df_input=df_oof_best_comp_clean, # <--- Используем OOF лучших
        component_names=COMPONENTS_TO_MODEL,
        lags=LAGS_TO_CREATE,
        windows=ROLLING_WINDOWS,
        feature_order_meta=META_FEATURE_ORDER # Ожидаем 35 признаков
    )
    # Обработка NaN после создания признаков
    X_train_meta_best_eng_clean = X_train_meta_best_eng.dropna()

    # Выравниваем таргет под очищенные признаки
    # Важно использовать train_values_aligned_meta, который был выровнен с df_oof_best_comp_clean
    y_train_meta_best_aligned = train_values_aligned_meta.loc[X_train_meta_best_eng_clean.index]

    print(f"Размер обучающей выборки с признаками (OOF лучших): {X_train_meta_best_eng_clean.shape}")
    print(f"Размер выровненного таргета: {y_train_meta_best_aligned.shape}")
    # Добавим проверку количества признаков
    if X_train_meta_best_eng_clean.shape[1] != len(META_FEATURE_ORDER):
         print(f"ПРЕДУПРЕЖДЕНИЕ: Количество признаков ({X_train_meta_best_eng_clean.shape[1]}) не совпадает с ожидаемым ({len(META_FEATURE_ORDER)})")

except Exception as e:
    print(f"Ошибка при создании обучающих признаков для мета-модели: {e}")
    X_train_meta_best_eng_clean = None
    y_train_meta_best_aligned = None

# --- 2. Создание признаков для ТЕСТОВОЙ выборки ---
if X_train_meta_best_eng_clean is not None:
    print("\nСоздание признаков для ТЕСТОВОЙ выборки (на прогнозах лучших моделей)...")
    try:
        max_lookback_best = max(max(LAGS_TO_CREATE, default=0), max(ROLLING_WINDOWS, default=0))
        # Используем историю из OOF лучших моделей
        if len(df_oof_best_comp_clean) >= max_lookback_best:
            history_for_test_best = df_oof_best_comp_clean.iloc[-max_lookback_best:]
        else:
            print(f"Предупреждение: Недостаточно OOF истории ({len(df_oof_best_comp_clean)}) для lookback ({max_lookback_best}).")
            history_for_test_best = df_oof_best_comp_clean.copy()

        # Объединяем историю OOF лучших и прогнозы лучших на тесте
        df_for_test_features_best = pd.concat([history_for_test_best, df_best_component_forecasts])
        # Генерируем признаки
        X_test_meta_best_eng_full = create_meta_features(
            df_input=df_for_test_features_best,
            component_names=COMPONENTS_TO_MODEL,
            lags=LAGS_TO_CREATE,
            windows=ROLLING_WINDOWS,
            feature_order_meta=META_FEATURE_ORDER # Ожидаем 35 признаков
        )
        # Оставляем только тестовый период
        X_test_meta_best_eng = X_test_meta_best_eng_full.loc[df_best_component_forecasts.index]

        # Проверка и обработка NaN в тесте
        if X_test_meta_best_eng.isnull().any().any():
            print("ВНИМАНИЕ: Обнаружены NaN в тестовых признаках (BestComp OOF). Заполняем медианой трейна...")
            for col in X_test_meta_best_eng.columns:
                if X_test_meta_best_eng[col].isnull().any():
                    median_val = X_train_meta_best_eng_clean[col].median()
                    X_test_meta_best_eng[col] = X_test_meta_best_eng[col].fillna(median_val)
            print(f"Кол-во NaN после заполнения: {X_test_meta_best_eng.isnull().sum().sum()}")
        else:
            print("NaN в тестовых признаках (BestComp OOF) не обнаружены.")

        print("\n--- Признаки для мета-модели на лучших OOF готовы ---")
        print("Созданы: X_train_meta_best_eng_clean, y_train_meta_best_aligned, X_test_meta_best_eng")
        print(f"Размер X_train_meta_best_eng_clean: {X_train_meta_best_eng_clean.shape}")
        print(f"Размер y_train_meta_best_aligned: {y_train_meta_best_aligned.shape}")
        print(f"Размер X_test_meta_best_eng: {X_test_meta_best_eng.shape}")

    except Exception as e:
        print(f"Ошибка при создании тестовых признаков для мета-модели: {e}")
        X_test_meta_best_eng = None
else:
     print("Пропуск создания тестовых признаков из-за ошибки на этапе создания обучающих.")
     X_test_meta_best_eng = None

print("\n--- Ячейка 18 завершена ---")

In [ ]:
print("\nЯчейка 19: Обучение и Оценка ВСЕХ Мета-моделей на Лучших OOF")

# --- ПРЕДПОЛАГАЕМ НАЛИЧИЕ ПЕРЕМЕННЫХ ---
# X_train_meta_best_eng_clean: DataFrame с 35 признаками из OOF лучших моделей (трейн)
# y_train_meta_best_aligned: Series с таргетом (агрегат)
# X_test_meta_best_eng: DataFrame с 35 признаками из прогнозов лучших моделей (тест)
# test_values: Series с фактом на тесте
# N_CV_SPLITS: int
# HORIZONS_TO_EVALUATE: list
# aggregation_rmse, all_horizon_rmse, final_forecasts
# RANDOM_SEED
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

# Переменные для хранения прогнозов этой ячейки
meta_forecasts_on_best = {}

# --- Масштабирование Признаков (для Ridge/Lasso) ---
print("Масштабирование признаков для Ridge/Lasso...")
scaler_meta_best = StandardScaler() # Новый scaler для этих данных
X_train_meta_best_scaled = scaler_meta_best.fit_transform(X_train_meta_best_eng_clean)
X_test_meta_best_scaled = scaler_meta_best.transform(X_test_meta_best_eng)
print("Масштабирование выполнено.")


# Инициализируем TS Splitter для CV
tscv_meta_best = TimeSeriesSplit(n_splits=N_CV_SPLITS)

# --- Meta-Ridge на Лучших OOF ---
model_name = "Meta-Ridge_on_BestComp"
print(f"\n--- Обучение и оценка: {model_name} ---")
try:
    alphas = np.logspace(-6, 6, 13)
    ridge_model = RidgeCV(alphas=alphas, store_cv_values=False, cv=None)
    ridge_model.fit(X_train_meta_best_scaled, y_train_meta_best_aligned) # На масштабированных
    print(f"  Лучший alpha: {ridge_model.alpha_:.6f}")
    y_pred = ridge_model.predict(X_test_meta_best_scaled) # На масштабированных
    meta_forecasts_on_best[model_name] = y_pred
    final_forecasts[model_name] = y_pred
    # --- Оценка (Общий RMSE) ---
    eval_len = min(len(y_pred), len(test_values))
    overall_rmse = np.sqrt(mean_squared_error(test_values[:eval_len], y_pred[:eval_len]))
    print(f"  Общий RMSE: {overall_rmse:.4f}")
    aggregation_rmse[model_name] = overall_rmse
    # --- Оценка (По горизонтам) ---
    horizon_results_list = []
    for h in HORIZONS_TO_EVALUATE:
        if h <= eval_len:
            rmse_h = np.sqrt(mean_squared_error(test_values[:h], y_pred[:h]))
            horizon_results_list.append({'model': model_name, 'horizon': h, 'rmse': rmse_h})
    if horizon_results_list:
        df_horizon = pd.DataFrame(horizon_results_list)
        all_horizon_rmse = all_horizon_rmse[all_horizon_rmse['model'] != model_name]
        all_horizon_rmse = pd.concat([all_horizon_rmse, df_horizon], ignore_index=True)
except Exception as e: print(f"  Ошибка {model_name}: {e}"); aggregation_rmse[model_name] = np.nan


# --- Meta-Lasso на Лучших OOF ---
model_name = "Meta-Lasso_on_BestComp"
print(f"\n--- Обучение и оценка: {model_name} ---")
try:
    lasso_model = LassoCV(cv=tscv_meta_best, random_state=RANDOM_SEED, max_iter=10000, n_jobs=-1, verbose=0)
    lasso_model.fit(X_train_meta_best_scaled, y_train_meta_best_aligned) # На масштабированных
    print(f"  Лучший alpha: {lasso_model.alpha_:.6f}")
    n_feat = np.sum(lasso_model.coef_ != 0); print(f"  Признаков отобрано: {n_feat}")
    y_pred = lasso_model.predict(X_test_meta_best_scaled) # На масштабированных
    meta_forecasts_on_best[model_name] = y_pred
    final_forecasts[model_name] = y_pred
    # --- Оценка ---
    eval_len = min(len(y_pred), len(test_values))
    overall_rmse = np.sqrt(mean_squared_error(test_values[:eval_len], y_pred[:eval_len]))
    print(f"  Общий RMSE: {overall_rmse:.4f}")
    aggregation_rmse[model_name] = overall_rmse
    horizon_results_list = []
    for h in HORIZONS_TO_EVALUATE:
        if h <= eval_len:
            rmse_h = np.sqrt(mean_squared_error(test_values[:h], y_pred[:h]))
            horizon_results_list.append({'model': model_name, 'horizon': h, 'rmse': rmse_h})
    if horizon_results_list:
        df_horizon = pd.DataFrame(horizon_results_list)
        all_horizon_rmse = all_horizon_rmse[all_horizon_rmse['model'] != model_name]
        all_horizon_rmse = pd.concat([all_horizon_rmse, df_horizon], ignore_index=True)
except Exception as e: print(f"  Ошибка {model_name}: {e}"); aggregation_rmse[model_name] = np.nan


# --- Meta-RandomForest на Лучших OOF ---
model_name = "Meta-RF_on_BestComp"
print(f"\n--- Обучение и оценка: {model_name} ---")
try:
    param_grid_rf = { # Используем ту же сетку, что и для RF_on_ARIMA
        'n_estimators': [100, 200], 'max_depth': [5, 10, None],
        'max_features': ['sqrt', 0.7], 'min_samples_leaf': [1, 3]
    }
    rf_model = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1)
    gs_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid_rf, scoring='neg_root_mean_squared_error',
                         cv=tscv_meta_best, n_jobs=-1, verbose=1, refit=True)
    print("Подбор параметров для Meta-RF (на лучших OOF)...")
    gs_rf.fit(X_train_meta_best_eng_clean, y_train_meta_best_aligned) # На НЕмасштабированных
    print(f"  Лучшие параметры: {gs_rf.best_params_}")
    print(f"  Лучший CV RMSE: {-gs_rf.best_score_:.4f}")
    best_rf = gs_rf.best_estimator_
    y_pred = best_rf.predict(X_test_meta_best_eng) # На НЕмасштабированных
    meta_forecasts_on_best[model_name] = y_pred
    final_forecasts[model_name] = y_pred
    # --- Оценка ---
    eval_len = min(len(y_pred), len(test_values))
    overall_rmse = np.sqrt(mean_squared_error(test_values[:eval_len], y_pred[:eval_len]))
    print(f"  Общий RMSE: {overall_rmse:.4f}")
    aggregation_rmse[model_name] = overall_rmse
    horizon_results_list = []
    for h in HORIZONS_TO_EVALUATE:
        if h <= eval_len:
            rmse_h = np.sqrt(mean_squared_error(test_values[:h], y_pred[:h]))
            horizon_results_list.append({'model': model_name, 'horizon': h, 'rmse': rmse_h})
    if horizon_results_list:
        df_horizon = pd.DataFrame(horizon_results_list)
        all_horizon_rmse = all_horizon_rmse[all_horizon_rmse['model'] != model_name]
        all_horizon_rmse = pd.concat([all_horizon_rmse, df_horizon], ignore_index=True)
except Exception as e: print(f"  Ошибка {model_name}: {e}"); aggregation_rmse[model_name] = np.nan


# --- Meta-XGBoost на Лучших OOF ---
model_name = "Meta-XGB_on_BestComp"
print(f"\n--- Обучение и оценка: {model_name} ---")
try:
    param_grid_xgb = { # Используем ту же сетку, что и для XGB_on_ARIMA
        'n_estimators': [50, 100], 'max_depth': [2, 3],
        'learning_rate': [0.1, 0.15], 'subsample': [0.8], 'colsample_bytree': [0.8]
    }
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=RANDOM_SEED, n_jobs=-1)
    gs_xgb = GridSearchCV(estimator=xgb_model, param_grid=param_grid_xgb, scoring='neg_root_mean_squared_error',
                          cv=tscv_meta_best, n_jobs=-1, verbose=1, refit=True)
    print("Подбор параметров для Meta-XGB (на лучших OOF)...")
    gs_xgb.fit(X_train_meta_best_eng_clean, y_train_meta_best_aligned) # На НЕмасштабированных
    print(f"  Лучшие параметры: {gs_xgb.best_params_}")
    print(f"  Лучший CV RMSE: {-gs_xgb.best_score_:.4f}")
    best_xgb = gs_xgb.best_estimator_
    y_pred = best_xgb.predict(X_test_meta_best_eng) # На НЕмасштабированных
    meta_forecasts_on_best[model_name] = y_pred
    final_forecasts[model_name] = y_pred
    # --- Оценка ---
    eval_len = min(len(y_pred), len(test_values))
    overall_rmse = np.sqrt(mean_squared_error(test_values[:eval_len], y_pred[:eval_len]))
    print(f"  Общий RMSE: {overall_rmse:.4f}")
    aggregation_rmse[model_name] = overall_rmse
    horizon_results_list = []
    for h in HORIZONS_TO_EVALUATE:
        if h <= eval_len:
            rmse_h = np.sqrt(mean_squared_error(test_values[:h], y_pred[:h]))
            horizon_results_list.append({'model': model_name, 'horizon': h, 'rmse': rmse_h})
    if horizon_results_list:
        df_horizon = pd.DataFrame(horizon_results_list)
        all_horizon_rmse = all_horizon_rmse[all_horizon_rmse['model'] != model_name]
        all_horizon_rmse = pd.concat([all_horizon_rmse, df_horizon], ignore_index=True)
except Exception as e: print(f"  Ошибка {model_name}: {e}"); aggregation_rmse[model_name] = np.nan


# --- Вывод Финальных Таблиц ---
print("\n--- Финальное сравнение RMSE (Общий RMSE) ---")
if 'aggregation_rmse' in locals() and aggregation_rmse:
    valid_keys = [k for k in aggregation_rmse if pd.notna(aggregation_rmse[k])]
    if valid_keys:
        sorted_rmse_final = {k: aggregation_rmse[k] for k in sorted(valid_keys, key=aggregation_rmse.get)}
        # Убедимся, что все модели есть в словаре перед выводом
        print("Модели в словаре:", list(aggregation_rmse.keys()))
        for model, rmse in sorted_rmse_final.items(): print(f"  {model}: {rmse:.4f}")
    else: print("Нет валидных RMSE для сортировки.")
else: print("Словарь aggregation_rmse пуст.")

print("\nФинальная таблица RMSE по горизонтам:")
if 'all_horizon_rmse' in locals() and not all_horizon_rmse.empty:
    try:
        pivot_rmse_final_final = all_horizon_rmse.pivot(index='horizon', columns='model', values='rmse')
        if 'sorted_rmse_final' in locals():
             ordered_columns_keys_final = list(sorted_rmse_final.keys())
             ordered_columns_final_final = [col for col in ordered_columns_keys_final if col in pivot_rmse_final_final.columns]
             remaining_cols_final_final = [col for col in pivot_rmse_final_final.columns if col not in ordered_columns_final_final]
             pivot_rmse_final_final = pivot_rmse_final_final[ordered_columns_final_final + remaining_cols_final_final]
        print(pivot_rmse_final_final.round(4))
    except Exception as e: print(f"Не удалось создать сводную таблицу: {e}")
else: print("Нет данных.")

print("\n--- Ячейка 19 (Все мета-модели на лучших OOF) завершена ---")

In [ ]:
# Используем даты из Baseline ARIMA в качестве общего индекса
dates = final_forecasts['Baseline ARIMA'].index
df = pd.DataFrame(final_forecasts, index=dates)

# Сохраняем в Excel
df.to_excel('Прогноз-43-компоненты-февраль23.xlsx', index=True, index_label='Date')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV # GridSearchCV остается для сравнения, если нужно, но мы его заменяем
from sklearn.metrics import mean_squared_error
import traceback

# Импорты для Hyperopt (если еще не были сделаны в этом ноутбуке/скрипте глобально)
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll.base import scope # Для scope.int

print("\nЯчейка 19: Обучение и Оценка ВСЕХ Мета-моделей на Лучших OOF (с Hyperopt для RF и XGB)")

# --- ПРЕДПОЛАГАЕМ НАЛИЧИЕ ПЕРЕМЕННЫХ ---
# X_train_meta_best_eng_clean: DataFrame (трейн признаки мета-модели)
# y_train_meta_best_aligned: Series (трейн таргет мета-модели)
# X_test_meta_best_eng: DataFrame (тест признаки мета-модели)
# test_values: Series (факт на тесте для оценки)
# N_CV_SPLITS: int (для TimeSeriesSplit)
# HORIZONS_TO_EVALUATE: list
# RANDOM_SEED: int
# MAX_EVALS_HYPEROPT_META: int (новое, количество итераций Hyperopt для мета-моделей, например, 30)

# Структуры для хранения результатов (предполагаем, что они инициализированы ранее)
# aggregation_rmse = {}
# all_horizon_rmse = pd.DataFrame(columns=['model', 'horizon', 'rmse'])
# final_forecasts = {}
# --- КОНЕЦ ПРЕДПОЛОЖЕНИЙ ---

MAX_EVALS_HYPEROPT_META=50

# Инициализация структур, если они не существуют
if 'aggregation_rmse' not in locals(): aggregation_rmse = {}
if 'all_horizon_rmse' not in locals(): all_horizon_rmse = pd.DataFrame(columns=['model', 'horizon', 'rmse'])
if 'final_forecasts' not in locals(): final_forecasts = {}
if 'MAX_EVALS_HYPEROPT_META' not in locals(): MAX_EVALS_HYPEROPT_META = 30 # Значение по умолчанию

# Переменные для хранения прогнозов этой ячейки
meta_forecasts_on_best = {}

# --- Масштабирование Признаков (для Ridge/Lasso) ---
print("Масштабирование признаков для Ridge/Lasso...")
scaler_meta_best = StandardScaler()
# Убедимся, что нет NaN перед масштабированием, иначе fit/transform может выдать ошибку или предупреждение
if X_train_meta_best_eng_clean.isnull().any().any():
    print("ПРЕДУПРЕЖДЕНИЕ: Обнаружены NaN в X_train_meta_best_eng_clean перед масштабированием. Это может вызвать проблемы.")
if X_test_meta_best_eng.isnull().any().any():
    print("ПРЕДУПРЕЖДЕНИЕ: Обнаружены NaN в X_test_meta_best_eng перед масштабированием. Это может вызвать проблемы.")

X_train_meta_best_scaled = scaler_meta_best.fit_transform(X_train_meta_best_eng_clean)
X_test_meta_best_scaled = scaler_meta_best.transform(X_test_meta_best_eng)
print("Масштабирование выполнено.")


# Инициализируем TS Splitter для CV мета-моделей
tscv_meta_best = TimeSeriesSplit(n_splits=N_CV_SPLITS)

# --- Meta-Ridge на Лучших OOF (без изменений, использует RidgeCV) ---
model_name_ridge = "Meta-Ridge_on_BestComp"
print(f"\n--- Обучение и оценка: {model_name_ridge} ---")
try:
    alphas_ridge = np.logspace(-6, 6, 20) # Можно увеличить количество альф
    # RidgeCV с TimeSeriesSplit требует, чтобы cv был итератором
    # RidgeCV сам делает кросс-валидацию, если cv=None (LOO) или указан int (KFold)
    # Для TimeSeriesSplit лучше использовать GridSearchCV или реализовать цикл вручную,
    # но RidgeCV довольно устойчив. Оставим как есть для простоты, но для строгости
    # можно было бы обернуть в цикл с tscv_meta_best.
    ridge_model_meta = RidgeCV(alphas=alphas_ridge, store_cv_values=False, cv=None) # cv=None для Generalized Cross-Validation
    ridge_model_meta.fit(X_train_meta_best_scaled, y_train_meta_best_aligned)
    print(f"  Лучший alpha: {ridge_model_meta.alpha_:.6f}")
    y_pred_ridge = ridge_model_meta.predict(X_test_meta_best_scaled)
    meta_forecasts_on_best[model_name_ridge] = y_pred_ridge
    final_forecasts[model_name_ridge] = y_pred_ridge
    eval_len_ridge = min(len(y_pred_ridge), len(test_values))
    overall_rmse_ridge = np.sqrt(mean_squared_error(test_values[:eval_len_ridge], y_pred_ridge[:eval_len_ridge]))
    print(f"  Общий RMSE: {overall_rmse_ridge:.4f}")
    aggregation_rmse[model_name_ridge] = overall_rmse_ridge
    horizon_results_list_ridge = []
    for h_r in HORIZONS_TO_EVALUATE:
        if h_r <= eval_len_ridge:
            rmse_h_r = np.sqrt(mean_squared_error(test_values[:h_r], y_pred_ridge[:h_r]))
            horizon_results_list_ridge.append({'model': model_name_ridge, 'horizon': h_r, 'rmse': rmse_h_r})
    if horizon_results_list_ridge:
        df_horizon_ridge = pd.DataFrame(horizon_results_list_ridge)
        all_horizon_rmse = all_horizon_rmse[all_horizon_rmse['model'] != model_name_ridge] # Удаляем старые записи
        all_horizon_rmse = pd.concat([all_horizon_rmse, df_horizon_ridge], ignore_index=True)
except Exception as e_r:
    print(f"  Ошибка {model_name_ridge}: {e_r}")
    aggregation_rmse[model_name_ridge] = np.nan
    traceback.print_exc()


# --- Meta-Lasso на Лучших OOF (без изменений, использует LassoCV) ---
model_name_lasso = "Meta-Lasso_on_BestComp"
print(f"\n--- Обучение и оценка: {model_name_lasso} ---")
try:
    # LassoCV хорошо работает с объектом TimeSeriesSplit
    lasso_model_meta = LassoCV(cv=tscv_meta_best, random_state=RANDOM_SEED, max_iter=20000, n_jobs=-1, verbose=0, n_alphas=100)
    lasso_model_meta.fit(X_train_meta_best_scaled, y_train_meta_best_aligned)
    print(f"  Лучший alpha: {lasso_model_meta.alpha_:.6f}")
    n_feat_lasso = np.sum(lasso_model_meta.coef_ != 0)
    print(f"  Признаков отобрано: {n_feat_lasso}")
    y_pred_lasso = lasso_model_meta.predict(X_test_meta_best_scaled)
    meta_forecasts_on_best[model_name_lasso] = y_pred_lasso
    final_forecasts[model_name_lasso] = y_pred_lasso
    eval_len_lasso = min(len(y_pred_lasso), len(test_values))
    overall_rmse_lasso = np.sqrt(mean_squared_error(test_values[:eval_len_lasso], y_pred_lasso[:eval_len_lasso]))
    print(f"  Общий RMSE: {overall_rmse_lasso:.4f}")
    aggregation_rmse[model_name_lasso] = overall_rmse_lasso
    horizon_results_list_lasso = []
    for h_l in HORIZONS_TO_EVALUATE:
        if h_l <= eval_len_lasso:
            rmse_h_l = np.sqrt(mean_squared_error(test_values[:h_l], y_pred_lasso[:h_l]))
            horizon_results_list_lasso.append({'model': model_name_lasso, 'horizon': h_l, 'rmse': rmse_h_l})
    if horizon_results_list_lasso:
        df_horizon_lasso = pd.DataFrame(horizon_results_list_lasso)
        all_horizon_rmse = all_horizon_rmse[all_horizon_rmse['model'] != model_name_lasso] # Удаляем старые записи
        all_horizon_rmse = pd.concat([all_horizon_rmse, df_horizon_lasso], ignore_index=True)
except Exception as e_l:
    print(f"  Ошибка {model_name_lasso}: {e_l}")
    aggregation_rmse[model_name_lasso] = np.nan
    traceback.print_exc()

# --- Определяем пространства поиска Hyperopt для мета-моделей ---
# Можно взять за основу пространства из Ячейки 14 или адаптировать их
rf_hyperopt_space_meta = {
    'n_estimators': scope.int(hp.quniform('n_estimators_rf_meta', 50, 250, 25)),
    'max_depth': hp.choice('max_depth_rf_meta', [None, scope.int(hp.quniform('max_depth_val_rf_meta', 3, 12, 1))]), # Немного уменьшил глубину для мета
    'max_features': hp.choice('max_features_rf_meta', ['sqrt', 'log2', hp.uniform('max_features_float_rf_meta', 0.3, 0.8)]),
    'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf_rf_meta', 1, 5, 1)),
    'min_samples_split': scope.int(hp.quniform('min_samples_split_rf_meta', 2, 10, 1))
}

xgb_hyperopt_space_meta = {
    'n_estimators': scope.int(hp.quniform('n_estimators_xgb_meta', 50, 300, 25)), # Чуть больше диапазон
    'max_depth': scope.int(hp.quniform('max_depth_xgb_meta', 2, 6, 1)), # Обычно мета-модели не требуют большой глубины
    'learning_rate': hp.loguniform('learning_rate_xgb_meta', np.log(0.01), np.log(0.2)),
    'subsample': hp.uniform('subsample_xgb_meta', 0.6, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree_xgb_meta', 0.6, 1.0),
    'gamma': hp.uniform('gamma_xgb_meta', 0, 0.5),
    'reg_alpha': hp.uniform('reg_alpha_xgb_meta', 0, 0.3),
    'reg_lambda': hp.uniform('reg_lambda_xgb_meta', 0, 0.3),
}

# --- Meta-RandomForest на Лучших OOF с Hyperopt ---
model_name_rf = "Meta-RF_on_BestComp"
print(f"\n--- Обучение и оценка: {model_name_rf} (с Hyperopt) ---")
try:
    def objective_rf_meta(params):
        current_params = params.copy()
        # Преобразование целочисленных параметров
        for p_name in ['n_estimators', 'max_depth', 'min_samples_leaf', 'min_samples_split']:
            if p_name in current_params and current_params[p_name] is not None:
                current_params[p_name] = int(current_params[p_name])

        model = RandomForestRegressor(**current_params, random_state=RANDOM_SEED, n_jobs=-1)
        cv_scores = []
        for train_idx, val_idx in tscv_meta_best.split(X_train_meta_best_eng_clean):
            X_cv_train, X_cv_val = X_train_meta_best_eng_clean.iloc[train_idx], X_train_meta_best_eng_clean.iloc[val_idx]
            y_cv_train, y_cv_val = y_train_meta_best_aligned.iloc[train_idx], y_train_meta_best_aligned.iloc[val_idx]
            if X_cv_train.empty or X_cv_val.empty: continue
            try:
                model.fit(X_cv_train, y_cv_train)
                preds = model.predict(X_cv_val)
                rmse = np.sqrt(mean_squared_error(y_cv_val, preds))
                cv_scores.append(rmse)
            except Exception: cv_scores.append(np.inf)
        if not cv_scores or all(s == np.inf for s in cv_scores): return {'loss': np.inf, 'status': STATUS_OK}
        avg_rmse = np.mean([s for s in cv_scores if s != np.inf])
        return {'loss': avg_rmse if not np.isnan(avg_rmse) else np.inf, 'status': STATUS_OK}

    trials_rf_meta = Trials()
    print(f"  Подбор параметров для {model_name_rf} с Hyperopt (max_evals={MAX_EVALS_HYPEROPT_META})...")
    best_params_rf_fmin = fmin(fn=objective_rf_meta,
                             space=rf_hyperopt_space_meta,
                             algo=tpe.suggest,
                             max_evals=MAX_EVALS_HYPEROPT_META,
                             trials=trials_rf_meta,
                             rstate=np.random.default_rng(RANDOM_SEED),
                             verbose=0) # verbose=1 для отладки

    actual_best_params_rf_meta = hyperopt.space_eval(rf_hyperopt_space_meta, best_params_rf_fmin)
    # Повторное преобразование целочисленных параметров после space_eval
    for p_name in ['n_estimators', 'max_depth', 'min_samples_leaf', 'min_samples_split']:
            if p_name in actual_best_params_rf_meta and actual_best_params_rf_meta[p_name] is not None:
                actual_best_params_rf_meta[p_name] = int(actual_best_params_rf_meta[p_name])

    best_loss_rf_meta = trials_rf_meta.best_trial['result']['loss'] if trials_rf_meta.best_trial else np.inf
    print(f"  Лучшие параметры: {actual_best_params_rf_meta}")
    print(f"  Лучший CV RMSE (Hyperopt): {best_loss_rf_meta:.4f}")

    final_rf_meta_model = RandomForestRegressor(**actual_best_params_rf_meta, random_state=RANDOM_SEED, n_jobs=-1)
    final_rf_meta_model.fit(X_train_meta_best_eng_clean, y_train_meta_best_aligned) # На НЕмасштабированных

    y_pred_rf = final_rf_meta_model.predict(X_test_meta_best_eng) # На НЕмасштабированных
    meta_forecasts_on_best[model_name_rf] = y_pred_rf
    final_forecasts[model_name_rf] = y_pred_rf

    eval_len_rf = min(len(y_pred_rf), len(test_values))
    overall_rmse_rf = np.sqrt(mean_squared_error(test_values[:eval_len_rf], y_pred_rf[:eval_len_rf]))
    print(f"  Общий RMSE: {overall_rmse_rf:.4f}")
    aggregation_rmse[model_name_rf] = overall_rmse_rf
    horizon_results_list_rf = []
    for h_rf in HORIZONS_TO_EVALUATE:
        if h_rf <= eval_len_rf:
            rmse_h_rf = np.sqrt(mean_squared_error(test_values[:h_rf], y_pred_rf[:h_rf]))
            horizon_results_list_rf.append({'model': model_name_rf, 'horizon': h_rf, 'rmse': rmse_h_rf})
    if horizon_results_list_rf:
        df_horizon_rf = pd.DataFrame(horizon_results_list_rf)
        all_horizon_rmse = all_horizon_rmse[all_horizon_rmse['model'] != model_name_rf]
        all_horizon_rmse = pd.concat([all_horizon_rmse, df_horizon_rf], ignore_index=True)
except Exception as e_rf:
    print(f"  Ошибка {model_name_rf}: {e_rf}")
    aggregation_rmse[model_name_rf] = np.nan
    traceback.print_exc()


# --- Meta-XGBoost на Лучших OOF с Hyperopt ---
model_name_xgb = "Meta-XGB_on_BestComp"
print(f"\n--- Обучение и оценка: {model_name_xgb} (с Hyperopt) ---")
try:
    def objective_xgb_meta(params):
        current_params = params.copy()
        # Преобразование целочисленных параметров
        for p_name in ['n_estimators', 'max_depth']: # Добавьте другие, если есть
            if p_name in current_params and current_params[p_name] is not None:
                current_params[p_name] = int(current_params[p_name])

        model = xgb.XGBRegressor(**current_params, objective='reg:squarederror', random_state=RANDOM_SEED, n_jobs=-1)
        cv_scores = []
        for train_idx, val_idx in tscv_meta_best.split(X_train_meta_best_eng_clean):
            X_cv_train, X_cv_val = X_train_meta_best_eng_clean.iloc[train_idx], X_train_meta_best_eng_clean.iloc[val_idx]
            y_cv_train, y_cv_val = y_train_meta_best_aligned.iloc[train_idx], y_train_meta_best_aligned.iloc[val_idx]
            if X_cv_train.empty or X_cv_val.empty: continue
            try:
                model.fit(X_cv_train, y_cv_train)
                preds = model.predict(X_cv_val)
                rmse = np.sqrt(mean_squared_error(y_cv_val, preds))
                cv_scores.append(rmse)
            except Exception: cv_scores.append(np.inf)
        if not cv_scores or all(s == np.inf for s in cv_scores): return {'loss': np.inf, 'status': STATUS_OK}
        avg_rmse = np.mean([s for s in cv_scores if s != np.inf])
        return {'loss': avg_rmse if not np.isnan(avg_rmse) else np.inf, 'status': STATUS_OK}

    trials_xgb_meta = Trials()
    print(f"  Подбор параметров для {model_name_xgb} с Hyperopt (max_evals={MAX_EVALS_HYPEROPT_META})...")
    best_params_xgb_fmin = fmin(fn=objective_xgb_meta,
                              space=xgb_hyperopt_space_meta,
                              algo=tpe.suggest,
                              max_evals=MAX_EVALS_HYPEROPT_META,
                              trials=trials_xgb_meta,
                              rstate=np.random.default_rng(RANDOM_SEED),
                              verbose=0)

    actual_best_params_xgb_meta = hyperopt.space_eval(xgb_hyperopt_space_meta, best_params_xgb_fmin)
    # Повторное преобразование целочисленных параметров
    for p_name in ['n_estimators', 'max_depth']:
            if p_name in actual_best_params_xgb_meta and actual_best_params_xgb_meta[p_name] is not None:
                actual_best_params_xgb_meta[p_name] = int(actual_best_params_xgb_meta[p_name])

    best_loss_xgb_meta = trials_xgb_meta.best_trial['result']['loss'] if trials_xgb_meta.best_trial else np.inf
    print(f"  Лучшие параметры: {actual_best_params_xgb_meta}")
    print(f"  Лучший CV RMSE (Hyperopt): {best_loss_xgb_meta:.4f}")

    final_xgb_meta_model = xgb.XGBRegressor(**actual_best_params_xgb_meta, objective='reg:squarederror', random_state=RANDOM_SEED, n_jobs=-1)
    final_xgb_meta_model.fit(X_train_meta_best_eng_clean, y_train_meta_best_aligned) # На НЕмасштабированных

    y_pred_xgb = final_xgb_meta_model.predict(X_test_meta_best_eng) # На НЕмасштабированных
    meta_forecasts_on_best[model_name_xgb] = y_pred_xgb
    final_forecasts[model_name_xgb] = y_pred_xgb

    eval_len_xgb = min(len(y_pred_xgb), len(test_values))
    overall_rmse_xgb = np.sqrt(mean_squared_error(test_values[:eval_len_xgb], y_pred_xgb[:eval_len_xgb]))
    print(f"  Общий RMSE: {overall_rmse_xgb:.4f}")
    aggregation_rmse[model_name_xgb] = overall_rmse_xgb
    horizon_results_list_xgb = []
    for h_xgb in HORIZONS_TO_EVALUATE:
        if h_xgb <= eval_len_xgb:
            rmse_h_xgb = np.sqrt(mean_squared_error(test_values[:h_xgb], y_pred_xgb[:h_xgb]))
            horizon_results_list_xgb.append({'model': model_name_xgb, 'horizon': h_xgb, 'rmse': rmse_h_xgb})
    if horizon_results_list_xgb:
        df_horizon_xgb = pd.DataFrame(horizon_results_list_xgb)
        all_horizon_rmse = all_horizon_rmse[all_horizon_rmse['model'] != model_name_xgb]
        all_horizon_rmse = pd.concat([all_horizon_rmse, df_horizon_xgb], ignore_index=True)
except Exception as e_xgb:
    print(f"  Ошибка {model_name_xgb}: {e_xgb}")
    aggregation_rmse[model_name_xgb] = np.nan
    traceback.print_exc()


# --- Вывод Финальных Таблиц ---
print("\n--- Финальное сравнение RMSE (Общий RMSE) ---")
if 'aggregation_rmse' in locals() and aggregation_rmse:
    # Фильтруем NaN значения перед сортировкой
    valid_rmse_items = {k: v for k, v in aggregation_rmse.items() if pd.notna(v)}
    if valid_rmse_items:
        sorted_rmse_final = dict(sorted(valid_rmse_items.items(), key=lambda item: item[1]))
        print("Модели в словаре aggregation_rmse:", list(aggregation_rmse.keys()))
        for model, rmse_val in sorted_rmse_final.items():
            print(f"  {model}: {rmse_val:.4f}")
    else:
        print("Нет валидных RMSE для сортировки в aggregation_rmse.")
else:
    print("Словарь aggregation_rmse не определен или пуст.")

print("\nФинальная таблица RMSE по горизонтам:")
if 'all_horizon_rmse' in locals() and not all_horizon_rmse.empty:
    try:
        # Удаляем дубликаты, если модель оценивалась несколько раз (например, при перезапусках)
        all_horizon_rmse_unique = all_horizon_rmse.drop_duplicates(subset=['model', 'horizon'], keep='last')
        pivot_rmse_final_final = all_horizon_rmse_unique.pivot(index='horizon', columns='model', values='rmse')
        # Сортируем колонки по общему RMSE из sorted_rmse_final, если он существует
        if 'sorted_rmse_final' in locals() and sorted_rmse_final:
             ordered_columns_keys_final = list(sorted_rmse_final.keys())
             # Включаем только те колонки, которые есть в pivot_rmse_final_final
             available_ordered_columns = [col for col in ordered_columns_keys_final if col in pivot_rmse_final_final.columns]
             remaining_cols_final_final = [col for col in pivot_rmse_final_final.columns if col not in available_ordered_columns]
             pivot_rmse_final_final = pivot_rmse_final_final[available_ordered_columns + remaining_cols_final_final]
        print(pivot_rmse_final_final.round(4))
    except Exception as e_pivot:
        print(f"Не удалось создать сводную таблицу RMSE по горизонтам: {e_pivot}")
        print("Данные в all_horizon_rmse:")
        print(all_horizon_rmse)
else:
    print("Нет данных в all_horizon_rmse для отображения таблицы по горизонтам.")

print("\n--- Ячейка 19 (Все мета-модели на лучших OOF с Hyperopt) завершена ---")

In [ ]:
# Используем даты из Baseline ARIMA в качестве общего индекса
dates = final_forecasts['Baseline ARIMA'].index
df = pd.DataFrame(final_forecasts, index=dates)

# Сохраняем в Excel
df.to_excel('Прогноз-43-компоненты-февраль23.xlsx', index=True, index_label='Date')